# Capstone Project - The Battle of the Neighborhoods (Week 2)

## Introduction: Business Problem

New york City is most populous city in USA . It's full of life, art, grit, food and culture. Due to its diversity and dynamism , opening a restaurant in New York city really requires thorough analysis about the neighborhood and understand location and cuisine type. If a person starts a restaurant in a lonely place , there are rare chances that it may succeed. Similar will be the case if one starts a restaurant in a place where there are too many restaurants. Understanding the neighborhood will help to find a balance.
This project helps to analyze the neighborhood of New York City and helps to identify the best location to open up a new restaurant business. Understanding similar businesses in the area will help to understand the following.

1. Understand the competitors - New York city is famous for its large variety of restaurants and food trucks. Its a highly competitive place and understanding the competitors is really helpful to identify whether similar business can be successful.
2. Analyze Cusines - Analyzing the neighborhood will reveal the different cusiness available in the area. Also it helps to identify the expected customers. If there is a specific cusine which is predominant in an area, we can expect more demand for that type of food and vice versa.
3. Analyze the crowd - Neighborhood analysis also reveals other types of business which are available in the area and helps to understand whether it is a crowded place or not.


## Data

Dataset for the project will be https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json This consists of 5 boroughs and 306 neighborhoods in New York City. Goal is to find the best location to start a restaurant in New York City. Foursquare API will be used to find restaurants and venues in the neighborhood.This dataset will be loaded and explored. Latitude and Longitude will be identified for the neighborhood which will be used to explore the nearby restaurants and popular places. Popular places depends on the time when the call is made. Neighboehood will be clustered using k-means. Each cluster will be compared to find the best location.



### Neighborhood Candidates

Let us find out the latitude and longitude of using specific, well known Foursquare api. Lets first download all dependencies.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


## Explore Data

Let's download and explore data

In [2]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
newyork_data

Data downloaded!


{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [3]:
 newyork_data['features']

[{'type': 'Feature',
  'id': 'nyu_2451_34572.1',
  'geometry': {'type': 'Point',
   'coordinates': [-73.84720052054902, 40.89470517661]},
  'geometry_name': 'geom',
  'properties': {'name': 'Wakefield',
   'stacked': 1,
   'annoline1': 'Wakefield',
   'annoline2': None,
   'annoline3': None,
   'annoangle': 0.0,
   'borough': 'Bronx',
   'bbox': [-73.84720052054902,
    40.89470517661,
    -73.84720052054902,
    40.89470517661]}},
 {'type': 'Feature',
  'id': 'nyu_2451_34572.2',
  'geometry': {'type': 'Point',
   'coordinates': [-73.82993910812398, 40.87429419303012]},
  'geometry_name': 'geom',
  'properties': {'name': 'Co-op City',
   'stacked': 2,
   'annoline1': 'Co-op',
   'annoline2': 'City',
   'annoline3': None,
   'annoangle': 0.0,
   'borough': 'Bronx',
   'bbox': [-73.82993910812398,
    40.87429419303012,
    -73.82993910812398,
    40.87429419303012]}},
 {'type': 'Feature',
  'id': 'nyu_2451_34572.3',
  'geometry': {'type': 'Point',
   'coordinates': [-73.82780644716412, 

In [4]:
pd.DataFrame( newyork_data['features'])

type                  id  \
0    Feature    nyu_2451_34572.1   
1    Feature    nyu_2451_34572.2   
2    Feature    nyu_2451_34572.3   
3    Feature    nyu_2451_34572.4   
4    Feature    nyu_2451_34572.5   
5    Feature    nyu_2451_34572.6   
6    Feature    nyu_2451_34572.7   
7    Feature    nyu_2451_34572.8   
8    Feature    nyu_2451_34572.9   
9    Feature   nyu_2451_34572.10   
10   Feature   nyu_2451_34572.11   
11   Feature   nyu_2451_34572.12   
12   Feature   nyu_2451_34572.13   
13   Feature   nyu_2451_34572.14   
14   Feature   nyu_2451_34572.15   
15   Feature   nyu_2451_34572.16   
16   Feature   nyu_2451_34572.17   
17   Feature   nyu_2451_34572.18   
18   Feature   nyu_2451_34572.19   
19   Feature   nyu_2451_34572.20   
20   Feature   nyu_2451_34572.21   
21   Feature   nyu_2451_34572.22   
22   Feature   nyu_2451_34572.23   
23   Feature   nyu_2451_34572.24   
24   Feature   nyu_2451_34572.25   
25   Feature   nyu_2451_34572.26   
26   Feature   nyu_2451_34572.27   
27   Feature   nyu_2451_34572.28   
28   Feature   nyu_2451_34572.29   
29   Feature   nyu_2451_34572.30   
30   Feature   nyu_2451_34572.31   
31   Feature   nyu_2451_34572.32   
32   Feature   nyu_2451_34572.33   
33   Feature   nyu_2451_34572.34   
34   Feature   nyu_2451_34572.35   
35   Feature   nyu_2451_34572.36   
36   Feature   nyu_2451_34572.37   
37   Feature   nyu_2451_34572.38   
38   Feature   nyu_2451_34572.39   
39   Feature   nyu_2451_34572.40   
40   Feature   nyu_2451_34572.41   
41   Feature   nyu_2451_34572.42   
42   Feature   nyu_2451_34572.43   
43   Feature   nyu_2451_34572.44   
44   Feature   nyu_2451_34572.45   
45   Feature   nyu_2451_34572.46   
46   Feature   nyu_2451_34572.47   
47   Feature   nyu_2451_34572.48   
48   Feature   nyu_2451_34572.49   
49   Feature   nyu_2451_34572.50   
50   Feature   nyu_2451_34572.51   
51   Feature   nyu_2451_34572.52   
52   Feature   nyu_2451_34572.53   
53   Feature   nyu_2451_34572.54   
54   Feature   nyu_2451_34572.55   
55   Feature   nyu_2451_34572.56   
56   Feature   nyu_2451_34572.57   
57   Feature   nyu_2451_34572.58   
58   Feature   nyu_2451_34572.59   
59   Feature   nyu_2451_34572.60   
60   Feature   nyu_2451_34572.61   
61   Feature   nyu_2451_34572.62   
62   Feature   nyu_2451_34572.63   
63   Feature   nyu_2451_34572.64   
64   Feature   nyu_2451_34572.65   
65   Feature   nyu_2451_34572.66   
66   Feature   nyu_2451_34572.67   
67   Feature   nyu_2451_34572.68   
68   Feature   nyu_2451_34572.69   
69   Feature   nyu_2451_34572.70   
70   Feature   nyu_2451_34572.71   
71   Feature   nyu_2451_34572.72   
72   Feature   nyu_2451_34572.73   
73   Feature   nyu_2451_34572.74   
74   Feature   nyu_2451_34572.75   
75   Feature   nyu_2451_34572.76   
76   Feature   nyu_2451_34572.77   
77   Feature   nyu_2451_34572.78   
78   Feature   nyu_2451_34572.79   
79   Feature   nyu_2451_34572.80   
80   Feature   nyu_2451_34572.81   
81   Feature   nyu_2451_34572.82   
82   Feature   nyu_2451_34572.83   
83   Feature   nyu_2451_34572.84   
84   Feature   nyu_2451_34572.85   
85   Feature   nyu_2451_34572.86   
86   Feature   nyu_2451_34572.87   
87   Feature   nyu_2451_34572.88   
88   Feature   nyu_2451_34572.89   
89   Feature   nyu_2451_34572.90   
90   Feature   nyu_2451_34572.91   
91   Feature   nyu_2451_34572.92   
92   Feature   nyu_2451_34572.93   
93   Feature   nyu_2451_34572.94   
94   Feature   nyu_2451_34572.95   
95   Feature   nyu_2451_34572.96   
96   Feature   nyu_2451_34572.97   
97   Feature   nyu_2451_34572.98   
98   Feature   nyu_2451_34572.99   
99   Feature  nyu_2451_34572.100   
100  Feature  nyu_2451_34572.101   
101  Feature  nyu_2451_34572.102   
102  Feature  nyu_2451_34572.103   
103  Feature  nyu_2451_34572.104   
104  Feature  nyu_2451_34572.105   
105  Feature  nyu_2451_34572.106   
106  Feature  nyu_2451_34572.107   
107  Feature  nyu_2451_34572.108   
108  Feature  nyu_2451_34572.109   
109  Feature  nyu_2451_34572.110   
110  Feature

Let us transform the data into dataframe.

In [5]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head()    
    

Borough Neighborhood   Latitude  Longitude
0   Bronx    Wakefield  40.894705 -73.847201
1   Bronx   Co-op City  40.874294 -73.829939
2   Bronx  Eastchester  40.887556 -73.827806
3   Bronx    Fieldston  40.895437 -73.905643
4   Bronx    Riverdale  40.890834 -73.912585

In [6]:
neighborhoods.groupby(["Borough"])["Neighborhood"].count().reset_index()

Borough  Neighborhood
0          Bronx            52
1       Brooklyn            70
2      Manhattan            40
3         Queens            81
4  Staten Island            63

Analyzing this data reveals that NY city has 5 boroughs -Bronx, Brooklyn, Manhattan,Queens and 	Staten Island.  We need to analyze neighborhood of each of these to find the best location to start restaurant.

###  Find latitude and Longitude

Geopy library can be used to get the latitude and longitude values of New York City

In [7]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Create a map of New York with neighborhoods 

In [8]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### Use Folium Library for Visualization
Folium is a great visualization library. Let us segment and cluster  the neighborhoods in each Borough

In [9]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()



Borough        Neighborhood   Latitude  Longitude
0  Manhattan         Marble Hill  40.876551 -73.910660
1  Manhattan           Chinatown  40.715618 -73.994279
2  Manhattan  Washington Heights  40.851903 -73.936900
3  Manhattan              Inwood  40.867684 -73.921210
4  Manhattan    Hamilton Heights  40.823604 -73.949688

In [10]:
Bronx_data = neighborhoods[neighborhoods['Borough'] == 'Bronx'].reset_index(drop=True)
Bronx_data.head()



Borough Neighborhood   Latitude  Longitude
0   Bronx    Wakefield  40.894705 -73.847201
1   Bronx   Co-op City  40.874294 -73.829939
2   Bronx  Eastchester  40.887556 -73.827806
3   Bronx    Fieldston  40.895437 -73.905643
4   Bronx    Riverdale  40.890834 -73.912585

In [11]:

Brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
Brooklyn_data.head()



Borough Neighborhood   Latitude  Longitude
0  Brooklyn    Bay Ridge  40.625801 -74.030621
1  Brooklyn  Bensonhurst  40.611009 -73.995180
2  Brooklyn  Sunset Park  40.645103 -74.010316
3  Brooklyn   Greenpoint  40.730201 -73.954241
4  Brooklyn    Gravesend  40.595260 -73.973471

In [12]:
Queens_data = neighborhoods[neighborhoods['Borough'] == 'Queens'].reset_index(drop=True)
Queens_data.head()


Borough     Neighborhood   Latitude  Longitude
0  Queens          Astoria  40.768509 -73.915654
1  Queens         Woodside  40.746349 -73.901842
2  Queens  Jackson Heights  40.751981 -73.882821
3  Queens         Elmhurst  40.744049 -73.881656
4  Queens     Howard Beach  40.654225 -73.838138

In [13]:

Staten_Island_data = neighborhoods[neighborhoods['Borough'] == 'Staten Island'].reset_index(drop=True)
Staten_Island_data.head()

Borough   Neighborhood   Latitude  Longitude
0  Staten Island     St. George  40.644982 -74.079353
1  Staten Island   New Brighton  40.640615 -74.087017
2  Staten Island      Stapleton  40.626928 -74.077902
3  Staten Island       Rosebank  40.615305 -74.069805
4  Staten Island  West Brighton  40.631879 -74.107182

Let's get the geographical coordinates of each Borough.

In [14]:
address_man = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address_man)
latitude_man = location.latitude
longitude_man = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude_man, longitude_man))

address_Bronx = 'Bronx, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address_Bronx)
latitude_Bronx = location.latitude
longitude_Bronx = location.longitude
print('The geograpical coordinate of Bronx are {}, {}.'.format(latitude_Bronx, longitude_Bronx))

address_Brooklyn = 'Brooklyn, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address_Brooklyn)
latitude_Brooklyn = location.latitude
longitude_Brooklyn = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude_Brooklyn, longitude_Brooklyn))

address_Queens = 'Queens, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address_Queens)
latitude_Queens= location.latitude
longitude_Queens = location.longitude
print('The geograpical coordinate of Queens are {}, {}.'.format(latitude_Queens, longitude_Queens))

address_Staten_Island = 'Staten Island, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address_Staten_Island)
latitude_Staten_Island = location.latitude
longitude_Staten_Island = location.longitude
print('The geograpical coordinate of Staten_Island are {}, {}.'.format(latitude_Staten_Island, longitude_Staten_Island))



The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.
The geograpical coordinate of Bronx are 40.8466508, -73.8785937.
The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.
The geograpical coordinate of Queens are 40.7498243, -73.7976337.
The geograpical coordinate of Staten_Island are 40.5834557, -74.1496048.


As we did with all of New York City, let's visualize each Borough and their neighborhoods .


In [15]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude_man, longitude_man], zoom_start=11)
 

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
map_manhattan

In [16]:
# create map of Manhattan using latitude and longitude values
map_Bronx = folium.Map(location=[latitude_Bronx, longitude_Bronx], zoom_start=11)
 

# add markers to map
for lat, lng, label in zip(Bronx_data['Latitude'], Bronx_data['Longitude'], Bronx_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Bronx)  
map_Bronx

In [17]:
# create map of Manhattan using latitude and longitude values
map_Brooklyn = folium.Map(location=[latitude_Brooklyn, longitude_Brooklyn], zoom_start=11)
 

# add markers to map
for lat, lng, label in zip(Brooklyn_data['Latitude'], Brooklyn_data['Longitude'], Brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Brooklyn)  
map_Brooklyn

In [18]:
# create map of Manhattan using latitude and longitude values
map_Queens = folium.Map(location=[latitude_Queens, longitude_Queens], zoom_start=11)
 

# add markers to map
for lat, lng, label in zip(Queens_data['Latitude'], Queens_data['Longitude'], Queens_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Queens)  
map_Queens

In [19]:
# create map of Manhattan using latitude and longitude values
map_Staten_Island = folium.Map(location=[latitude_Staten_Island, longitude_Staten_Island], zoom_start=11)
 

# add markers to map
for lat, lng, label in zip(Staten_Island_data['Latitude'], Staten_Island_data['Longitude'], Staten_Island_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Staten_Island)  
map_Staten_Island

### Using FourSquare API to  explore and Segment

Let us define the Foursquare API credentials and version to explore the neighborhoods and segment them.

In [20]:
CLIENT_ID = 'MF4LH1MWNCZ4SJO3X0UADY5O02ODRKRLF4SGPDJD4RJWIX0B' # your Foursquare ID
CLIENT_SECRET = '3UE1L2LSAVVM3YGAGRJSGPLSADRY305NPXTES0UKAQPYHH5I' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MF4LH1MWNCZ4SJO3X0UADY5O02ODRKRLF4SGPDJD4RJWIX0B
CLIENT_SECRET:3UE1L2LSAVVM3YGAGRJSGPLSADRY305NPXTES0UKAQPYHH5I


Let's create a function to explore all the neighborhoods in each Borough

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:

manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [23]:
Bronx_venues = getNearbyVenues(names=Bronx_data['Neighborhood'],
                                   latitudes=Bronx_data['Latitude'],
                                   longitudes=Bronx_data['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Claremont Village
Concourse Village
Mount Eden
Mount Hope
Bronxdale
Allerton
Kingsbridge Heights


In [24]:
Brooklyn_venues = getNearbyVenues(names=Brooklyn_data['Neighborhood'],
                                   latitudes=Brooklyn_data['Latitude'],
                                   longitudes=Brooklyn_data['Longitude']
                                  )

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


In [28]:
Queens_venues = getNearbyVenues(names=Queens_data['Neighborhood'],
                                   latitudes=Queens_data['Latitude'],
                                   longitudes=Queens_data['Longitude']
                                  )

Astoria
Woodside
Jackson Heights
Elmhurst
Howard Beach
Corona
Forest Hills
Kew Gardens
Richmond Hill
Flushing
Long Island City
Sunnyside
East Elmhurst
Maspeth
Ridgewood
Glendale
Rego Park
Woodhaven
Ozone Park
South Ozone Park
College Point
Whitestone
Bayside
Auburndale
Little Neck
Douglaston
Glen Oaks
Bellerose
Kew Gardens Hills
Fresh Meadows
Briarwood
Jamaica Center
Oakland Gardens
Queens Village
Hollis
South Jamaica
St. Albans
Rochdale
Springfield Gardens
Cambria Heights
Rosedale
Far Rockaway
Broad Channel
Breezy Point
Steinway
Beechhurst
Bay Terrace
Edgemere
Arverne
Rockaway Beach
Neponsit
Murray Hill
Floral Park
Holliswood
Jamaica Estates
Queensboro Hill
Hillcrest
Ravenswood
Lindenwood
Laurelton
Lefrak City
Belle Harbor
Rockaway Park
Somerville
Brookville
Bellaire
North Corona
Forest Hills Gardens
Jamaica Hills
Utopia
Pomonok
Astoria Heights
Hunters Point
Sunnyside Gardens
Blissville
Roxbury
Middle Village
Malba
Hammels
Bayswater
Queensbridge


In [26]:
Staten_Island_venues = getNearbyVenues(names=Staten_Island_data['Neighborhood'],
                                   latitudes=Staten_Island_data['Latitude'],
                                   longitudes=Staten_Island_data['Longitude']
                                  )

St. George
New Brighton
Stapleton
Rosebank
West Brighton
Grymes Hill
Todt Hill
South Beach
Port Richmond
Mariner's Harbor
Port Ivory
Castleton Corners
New Springville
Travis
New Dorp
Oakwood
Great Kills
Eltingville
Annadale
Woodrow
Tottenville
Tompkinsville
Silver Lake
Sunnyside
Park Hill
Westerleigh
Graniteville
Arlington
Arrochar
Grasmere
Old Town
Dongan Hills
Midland Beach
Grant City
New Dorp Beach
Bay Terrace
Huguenot
Pleasant Plains
Butler Manor
Charleston
Rossville
Arden Heights
Greenridge
Heartland Village
Chelsea
Bloomfield
Bulls Head
Richmond Town
Shore Acres
Clifton
Concord
Emerson Hill
Randall Manor
Howland Hook
Elm Park
Manor Heights
Willowbrook
Sandy Ground
Egbertville
Prince's Bay
Lighthouse Hill
Richmond Valley
Fox Hills


Let's check how many venues were returned for each neighborhood

In [30]:
manhattan_venues.groupby('Neighborhood').count()

Neighborhood Latitude  Neighborhood Longitude  Venue  \
Neighborhood                                                                
Battery Park City                       92                      92     92   
Carnegie Hill                           89                      89     89   
Central Harlem                          44                      44     44   
Chelsea                                100                     100    100   
Chinatown                              100                     100    100   
Civic Center                           100                     100    100   
Clinton                                100                     100    100   
East Harlem                             42                      42     42   
East Village                           100                     100    100   
Financial District                     100                     100    100   
Flatiron                               100                     100    100   
Gramercy                                91                      91     91   
Greenwich Village                      100                     100    100   
Hamilton Heights                        62                      62     62   
Hudson Yards                            74                      74     74   
Inwood                                  53                      53     53   
Lenox Hill                             100                     100    100   
Lincoln Square                          95                      95     95   
Little Italy                           100                     100    100   
Lower East Side                         50                      50     50   
Manhattan Valley                        48                      48     48   
Manhattanville                          47                      47     47   
Marble Hill                             24                      24     24   
Midtown                                100                     100    100   
Midtown South                          100                     100    100   
Morningside Heights                     43                      43     43   
Murray Hill                            100                     100    100   
Noho                                   100                     100    100   
Roosevelt Island                        31                      31     31   
Soho                                   100                     100    100   
Stuyvesant Town                         19                      19     19   
Sutton Place                           100                     100    100   
Tribeca                                 92                      92     92   
Tudor City                              81                      81     81   
Turtle Bay                             100                     100    100   
Upper East Side                        100                     100    100   
Upper West Side                         95                      95     95   
Washington Heights                      87                      87     87   
West Village                           100                     100    100   
Yorkville                              100                     100    100   

                     Venue Latitude  Venue Longitude  Venue Category  
Neighborhood                                                          
Battery Park City                92               92              92  
Carnegie Hill                    89               89              89  
Central Harlem                   44               44              44  
Chelsea                         100              100             100  
Chinatown                       100              100             100  
Civic Center                    100              100             100  
Clinton                         100              100             100  
East Harlem                      42               42              42  
East Village                    100              100             100  
Finan

We can analyze how many unique categories can be identified from the venues.

In [31]:
print('There are {} uniques categories in Manhattan.'.format(len(manhattan_venues['Venue Category'].unique())))
print('There are {} uniques categories in Bronx.'.format(len(Bronx_venues['Venue Category'].unique())))

print('There are {} uniques categories in Brooklyn.'.format(len(Brooklyn_venues['Venue Category'].unique())))
print('There are {} uniques categories in Queens.'.format(len(Queens_venues['Venue Category'].unique())))
print('There are {} uniques categories in Staten Island.'.format(len(Staten_Island_venues['Venue Category'].unique())))

There are 332 uniques categories in Manhattan.
There are 169 uniques categories in Bronx.
There are 288 uniques categories in Brooklyn.
There are 274 uniques categories in Queens.
There are 179 uniques categories in Staten Island.


## Analyze each neighborhood

In [32]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

Neighborhood  Accessories Store  Acupuncturist  Adult Boutique  \
0  Marble Hill                  0              0               0   
1  Marble Hill                  0              0               0   
2  Marble Hill                  0              0               0   
3  Marble Hill                  0              0               0   
4  Marble Hill                  0              0               0   

   Afghan Restaurant  African Restaurant  American Restaurant  Antique Shop  \
0                  0                   0                    0             0   
1                  0                   0                    0             0   
2                  0                   0                    0             0   
3                  0                   0                    0             0   
4                  0                   0                    0             0   

   Arepa Restaurant  Argentinian Restaurant  Art Gallery  Art Museum  \
0                 0                       0            0           0   
1                 0                       0            0           0   
2                 0                       0            0           0   
3                 0                       0            0           0   
4                 0                       0            0           0   

   Arts & Crafts Store  Asian Restaurant  Athletics & Sports  Auditorium  \
0                    0                 0                   0           0   
1                    0                 0                   0           0   
2                    0                 0                   0           0   
3                    0                 0                   0           0   
4                    0                 0                   0           0   

   Australian Restaurant  Austrian Restaurant  Auto Workshop  BBQ Joint  \
0                      0                    0              0          0   
1                      0                    0              0          0   
2                      0                    0              0          0   
3                      0                    0              0          0   
4                      0                    0              0          0   

   Baby Store  Bagel Shop  Bakery  Bank  Bar  Basketball Court  Beer Bar  \
0           0           0       0     0    0                 0         0   
1           0           0       0     0    0                 0         0   
2           0           0       0     0    0                 0         0   
3           0           0       0     0    0                 0         0   
4           0           0       0     0    0                 0         0   

   Beer Garden  Beer Store  Big Box Store  Bike Rental / Bike Share  \
0            0           0              0                         0   
1            0           0              0                         0   
2            0           0              0                         0   
3            0           0              0                         0   
4            0           0              0                         0   

   Bike Shop  Bike Trail  Bistro  Board Shop  Boat or Ferry  Bookstore  \
0          0           0       0           0              0          0   
1          0           0       0           0              0          0   
2          0           0       0           0              0          0   
3          0           0       0           0              0          0   
4          0           0       0           0              0          0   

   Boutique  Boxing Gym  Brazilian Restaurant  Breakfast Spot  Bridal Shop  \
0         0           0                     0               0            0   
1         0           0                     0               0            0   
2         0           0                     0               0            0   
3         0           0                     0               0            0   
4         0           0                     0             

In [33]:
# one hot encoding
Bronx_onehot = pd.get_dummies(Bronx_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Bronx_onehot['Neighborhood'] = Bronx_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Bronx_onehot.columns[-1]] + list(Bronx_onehot.columns[:-1])
Bronx_onehot = Bronx_onehot[fixed_columns]

Bronx_onehot.head()

Neighborhood  Accessories Store  African Restaurant  American Restaurant  \
0    Wakefield                  0                   0                    0   
1    Wakefield                  0                   0                    0   
2    Wakefield                  0                   0                    0   
3    Wakefield                  0                   0                    0   
4    Wakefield                  0                   0                    0   

   Arcade  Arepa Restaurant  Art Gallery  Art Museum  Arts & Crafts Store  \
0       0                 0            0           0                    0   
1       0                 0            0           0                    0   
2       0                 0            0           0                    0   
3       0                 0            0           0                    0   
4       0                 0            0           0                    0   

   Asian Restaurant  Athletics & Sports  Automotive Shop  BBQ Joint  \
0                 0                   0                0          0   
1                 0                   0                0          0   
2                 0                   0                0          0   
3                 0                   0                0          0   
4                 0                   0                0          0   

   Bagel Shop  Bakery  Bank  Bar  Baseball Field  Basketball Court  Beer Bar  \
0           0       0     0    0               0                 0         0   
1           0       0     0    0               0                 0         0   
2           0       0     0    0               0                 0         0   
3           0       0     0    0               0                 0         0   
4           0       0     0    0               0                 0         0   

   Board Shop  Boat or Ferry  Bookstore  Bowling Alley  Breakfast Spot  \
0           0              0          0              0               0   
1           0              0          0              0               0   
2           0              0          0              0               0   
3           0              0          0              0               0   
4           0              0          0              0               0   

   Brewery  Buffet  Building  Burger Joint  Burrito Place  Bus Line  \
0        0       0         0             0              0         0   
1        0       0         0             0              0         0   
2        0       0         0             0              0         0   
3        0       0         0             0              0         0   
4        0       0         0             0              0         0   

   Bus Station  Bus Stop  Café  Candy Store  Caribbean Restaurant  \
0            0         0     0            0                     0   
1            0         0     0            0                     0   
2            0         0     0            0                     0   
3            0         0     0            0                     0   
4            0         0     0            0                     0   

   Check Cashing Service  Cheese Shop  Chinese Restaurant  Clothing Store  \
0                      0            0                   0               0   
1                      0            0                   0               0   
2                      0            0                   0               0   
3                      0            0                   0               0   
4                      0            0                   0               0   

   Coffee Shop  Comfort Food Restaurant  Convenience Store  Cosmetics Shop  \
0            0                        0                  0               0   
1            0                        0                  0               0   
2            0                        0                  0               0   
3            0                        0                  0               0   
4            0        

In [34]:
# one hot encoding
Brooklyn_onehot = pd.get_dummies(Brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Brooklyn_onehot['Neighborhood'] = Brooklyn_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Brooklyn_onehot.columns[-1]] + list(Brooklyn_onehot.columns[:-1])
Brooklyn_onehot = Brooklyn_onehot[fixed_columns]

Brooklyn_onehot.head()

Yoga Studio  ATM  Accessories Store  Airport Terminal  American Restaurant  \
0            0    0                  0                 0                    0   
1            0    0                  0                 0                    0   
2            0    0                  0                 0                    0   
3            0    0                  0                 0                    0   
4            0    0                  0                 0                    0   

   Antique Shop  Arepa Restaurant  Argentinian Restaurant  Art Gallery  \
0             0                 0                       0            0   
1             0                 0                       0            0   
2             0                 0                       0            0   
3             0                 0                       0            0   
4             0                 0                       0            0   

   Art Museum  Arts & Crafts Store  Arts & Entertainment  Asian Restaurant  \
0           0                    0                     0                 0   
1           0                    0                     0                 0   
2           0                    0                     0                 0   
3           0                    0                     0                 0   
4           0                    0                     0                 0   

   Athletics & Sports  Auto Garage  BBQ Joint  Bagel Shop  Bakery  Bank  Bar  \
0                   0            0          0           0       0     0    0   
1                   0            0          0           1       0     0    0   
2                   0            0          0           0       0     0    0   
3                   0            0          0           0       0     0    0   
4                   0            0          0           0       0     0    0   

   Baseball Field  Baseball Stadium  Basketball Court  Beach  Beer Bar  \
0               0                 0                 0      0         0   
1               0                 0                 0      0         0   
2               0                 0                 0      0         0   
3               0                 0                 0      0         0   
4               0                 0                 0      0         0   

   Beer Garden  Beer Store  Big Box Store  Bike Rental / Bike Share  \
0            0           0              0                         0   
1            0           0              0                         0   
2            0           0              0                         0   
3            0           0              0                         0   
4            0           0              0                         0   

   Bike Shop  Bistro  Board Shop  Boat or Ferry  Bookstore  Boutique  \
0          0       0           0              0          0         0   
1          0       0           0              0          0         0   
2          0       0           0              0          0         0   
3          0       0           0              0          0         0   
4          0       0           0              0          0         0   

   Boxing Gym  Brazilian Restaurant  Breakfast Spot  Brewery  Bridge  \
0           0                     0               0        0       0   
1           0                     0               0        0       0   
2           0                     0               1        0       0   
3           0                     0               0        0       0   
4           0                     0               0        0       0   

   Bubble Tea Shop  Buffet  Burger Joint  Burmese Restaurant  Burrito Place  \
0                0       0             0                   0              0   
1                0       0             0                   0              0   
2                0       0             0                   0              0   
3                0       0             0                   0       

In [35]:
# one hot encoding
Queens_onehot = pd.get_dummies(Queens_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Queens_onehot['Neighborhood'] = Queens_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Queens_onehot.columns[-1]] + list(Queens_onehot.columns[:-1])
Queens_onehot = Queens_onehot[fixed_columns]

Queens_onehot.head()

Yoga Studio  Afghan Restaurant  American Restaurant  Arepa Restaurant  \
0            0                  0                    0                 0   
1            0                  0                    0                 0   
2            0                  0                    0                 0   
3            0                  0                    0                 0   
4            0                  0                    0                 0   

   Argentinian Restaurant  Art Gallery  Art Museum  Arts & Crafts Store  \
0                       0            0           0                    0   
1                       0            0           0                    0   
2                       0            0           0                    0   
3                       0            0           0                    0   
4                       0            0           0                    0   

   Arts & Entertainment  Asian Restaurant  Athletics & Sports  Auto Workshop  \
0                     0                 0                   0              0   
1                     0                 0                   0              0   
2                     0                 0                   0              0   
3                     0                 0                   0              0   
4                     0                 0                   0              0   

   Automotive Shop  BBQ Joint  Baby Store  Bagel Shop  Bakery  Bank  Bar  \
0                0          0           0           0       0     0    0   
1                0          0           0           0       0     0    0   
2                0          0           0           0       0     0    0   
3                0          0           0           0       0     0    0   
4                0          0           0           0       0     0    0   

   Baseball Field  Basketball Court  Bath House  Beach  Beach Bar  \
0               0                 0           0      0          0   
1               0                 0           0      0          0   
2               0                 0           0      0          0   
3               0                 0           0      0          0   
4               0                 0           0      0          0   

   Beer Garden  Big Box Store  Bike Trail  Bistro  Board Shop  Bookstore  \
0            0              0           0       0           0          0   
1            0              0           0       0           0          0   
2            0              0           0       0           0          0   
3            0              0           0       0           0          0   
4            0              0           0       0           0          0   

   Boutique  Bowling Alley  Boxing Gym  Brazilian Restaurant  Breakfast Spot  \
0         0              0           0                     1               0   
1         0              0           0                     0               0   
2         0              0           0                     0               0   
3         0              0           0                     0               0   
4         0              0           0                     0               0   

   Brewery  Bubble Tea Shop  Buffet  Building  Burger Joint  Burrito Place  \
0        0                0       0         0             0              0   
1        0                0       0         0             0              0   
2        0                0       0         0             0              0   
3        0                0       0         0             0              0   
4        0                0       0         0             0              0   

   Bus Line  Bus Station  Bus Stop  Butcher  Café  Cajun / Creole Restaurant  \
0         0            0         0        0     0                          0   
1         0            0         0        0     0                          0   
2         0            0         0        0     0                          0   
3         0    

In [36]:
# one hot encoding
Staten_Island_onehot = pd.get_dummies(Staten_Island_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Staten_Island_onehot['Neighborhood'] = Staten_Island_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Staten_Island_onehot.columns[-1]] + list(Staten_Island_onehot.columns[:-1])
Staten_Island_onehot = Staten_Island_onehot[fixed_columns]

Staten_Island_onehot.head()

Neighborhood  ATM  American Restaurant  Antique Shop  Arcade  Art Gallery  \
0   St. George    0                    0             0       0            0   
1   St. George    0                    0             0       0            0   
2   St. George    0                    0             0       0            0   
3   St. George    0                    0             0       0            0   
4   St. George    0                    0             0       0            0   

   Art Museum  Arts & Crafts Store  Asian Restaurant  Athletics & Sports  \
0           0                    0                 0                   0   
1           0                    0                 0                   0   
2           0                    0                 0                   0   
3           0                    0                 0                   0   
4           0                    0                 0                   0   

   BBQ Joint  Bagel Shop  Bakery  Bank  Bar  Baseball Field  Baseball Stadium  \
0          0           0       0     0    0               0                 0   
1          0           0       0     0    0               0                 0   
2          0           0       0     0    0               0                 0   
3          0           0       0     0    0               0                 0   
4          0           0       0     0    0               0                 1   

   Basketball Court  Beach  Beer Bar  Big Box Store  Board Shop  \
0                 0      0         0              0           0   
1                 0      0         0              0           0   
2                 0      0         0              0           0   
3                 0      0         0              0           0   
4                 0      0         0              0           0   

   Boat or Ferry  Bookstore  Bowling Alley  Breakfast Spot  Brewery  \
0              0          0              0               0        0   
1              0          0              0               0        0   
2              0          0              0               0        0   
3              0          0              0               0        0   
4              0          0              0               0        0   

   Bridal Shop  Bubble Tea Shop  Burger Joint  Bus Station  Bus Stop  \
0            0                0             0            0         0   
1            0                0             0            0         0   
2            0                0             0            0         0   
3            0                0             1            0         0   
4            0                0             0            0         0   

   Business Service  Butcher  Café  Cajun / Creole Restaurant  Campground  \
0                 0        0     0                          0           0   
1                 0        0     0                          0           0   
2                 0        0     0                          0           0   
3                 0        0     0                          0           0   
4                 0        0     0                          0           0   

   Caribbean Restaurant  Check Cashing Service  Chinese Restaurant  \
0                     0                      0                   0   
1                     0                      0                   0   
2                     0                      0                   0   
3                     0                      0                   0   
4                     0                      0                   0   

   Clothing Store  Coffee Shop  Comedy Club  Construction & Landscaping  \
0               0            0            0                           0   
1               0            0            0                           0   
2               0            0            0                           0   
3               0            0            0                           0   
4               0            0            0                  

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [37]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped.head()

Neighborhood  Accessories Store  Acupuncturist  Adult Boutique  \
0  Battery Park City                0.0            0.0             0.0   
1      Carnegie Hill                0.0            0.0             0.0   
2     Central Harlem                0.0            0.0             0.0   
3            Chelsea                0.0            0.0             0.0   
4          Chinatown                0.0            0.0             0.0   

   Afghan Restaurant  African Restaurant  American Restaurant  Antique Shop  \
0                0.0            0.000000             0.010870           0.0   
1                0.0            0.000000             0.011236           0.0   
2                0.0            0.068182             0.045455           0.0   
3                0.0            0.000000             0.040000           0.0   
4                0.0            0.000000             0.040000           0.0   

   Arepa Restaurant  Argentinian Restaurant  Art Gallery  Art Museum  \
0               0.0                0.000000     0.000000    0.000000   
1               0.0                0.011236     0.000000    0.022472   
2               0.0                0.000000     0.022727    0.000000   
3               0.0                0.000000     0.040000    0.000000   
4               0.0                0.000000     0.000000    0.000000   

   Arts & Crafts Store  Asian Restaurant  Athletics & Sports  Auditorium  \
0                  0.0              0.00             0.01087     0.01087   
1                  0.0              0.00             0.00000     0.00000   
2                  0.0              0.00             0.00000     0.00000   
3                  0.0              0.01             0.00000     0.00000   
4                  0.0              0.02             0.00000     0.00000   

   Australian Restaurant  Austrian Restaurant  Auto Workshop  BBQ Joint  \
0                    0.0                 0.00            0.0   0.021739   
1                    0.0                 0.00            0.0   0.000000   
2                    0.0                 0.00            0.0   0.022727   
3                    0.0                 0.00            0.0   0.000000   
4                    0.0                 0.01            0.0   0.000000   

   Baby Store  Bagel Shop    Bakery  Bank       Bar  Basketball Court  \
0         0.0    0.000000  0.010870   0.0  0.010870               0.0   
1         0.0    0.011236  0.022472   0.0  0.033708               0.0   
2         0.0    0.022727  0.000000   0.0  0.045455               0.0   
3         0.0    0.000000  0.050000   0.0  0.010000               0.0   
4         0.0    0.000000  0.080000   0.0  0.020000               0.0   

   Beer Bar  Beer Garden  Beer Store  Big Box Store  Bike Rental / Bike Share  \
0  0.000000     0.021739         0.0            0.0                       0.0   
1  0.000000     0.000000         0.0            0.0                       0.0   
2  0.022727     0.000000         0.0            0.0                       0.0   
3  0.010000     0.000000         0.0            0.0                       0.0   
4  0.000000     0.000000         0.0            0.0                       0.0   

   Bike Shop  Bike Trail   Bistro  Board Shop  Boat or Ferry  Bookstore  \
0        0.0         0.0  0.01087         0.0       0.021739   0.000000   
1        0.0         0.0  0.00000         0.0       0.000000   0.033708   
2        0.0         0.0  0.00000         0.0       0.000000   0.022727   
3        0.0         0.0  0.00000         0.0       0.000000   0.020000   
4        0.0         0.0  0.00000         0.0       0.000000   0.000000   

   Boutique  Boxing Gym  Brazilian Restaurant  Breakfast Spot  Bridal Shop  \
0  0.010870         0.0                   0.0            0.00          0.0   
1  0.000000         0.0                   0.0            0.00          0.0   
2  0.022727         0.0                   0.0            0.00          0.0   
3  0.000000         0.0                   0.0            0

In [38]:
Bronx_grouped = Bronx_onehot.groupby('Neighborhood').mean().reset_index()
Bronx_grouped.head()

Neighborhood  Accessories Store  African Restaurant  American Restaurant  \
0      Allerton                0.0                 0.0                 0.00   
1    Baychester                0.0                 0.0                 0.00   
2  Bedford Park                0.0                 0.0                 0.00   
3       Belmont                0.0                 0.0                 0.01   
4     Bronxdale                0.0                 0.0                 0.00   

     Arcade  Arepa Restaurant  Art Gallery  Art Museum  Arts & Crafts Store  \
0  0.000000               0.0          0.0         0.0                  0.0   
1  0.041667               0.0          0.0         0.0                  0.0   
2  0.000000               0.0          0.0         0.0                  0.0   
3  0.000000               0.0          0.0         0.0                  0.0   
4  0.000000               0.0          0.0         0.0                  0.0   

   Asian Restaurant  Athletics & Sports  Automotive Shop  BBQ Joint  \
0               0.0                 0.0              0.0        0.0   
1               0.0                 0.0              0.0        0.0   
2               0.0                 0.0              0.0        0.0   
3               0.0                 0.0              0.0        0.0   
4               0.0                 0.0              0.0        0.0   

   Bagel Shop    Bakery      Bank   Bar  Baseball Field  Basketball Court  \
0         0.0  0.000000  0.000000  0.00        0.000000               0.0   
1         0.0  0.000000  0.041667  0.00        0.041667               0.0   
2         0.0  0.027778  0.000000  0.00        0.027778               0.0   
3         0.0  0.050000  0.030000  0.02        0.000000               0.0   
4         0.0  0.000000  0.083333  0.00        0.000000               0.0   

   Beer Bar  Board Shop  Boat or Ferry  Bookstore  Bowling Alley  \
0       0.0         0.0            0.0        0.0            0.0   
1       0.0         0.0            0.0        0.0            0.0   
2       0.0         0.0            0.0        0.0            0.0   
3       0.0         0.0            0.0        0.0            0.0   
4       0.0         0.0            0.0        0.0            0.0   

   Breakfast Spot  Brewery  Buffet  Building  Burger Joint  Burrito Place  \
0        0.037037      0.0     0.0       0.0      0.000000            0.0   
1        0.000000      0.0     0.0       0.0      0.041667            0.0   
2        0.000000      0.0     0.0       0.0      0.000000            0.0   
3        0.000000      0.0     0.0       0.0      0.000000            0.0   
4        0.083333      0.0     0.0       0.0      0.000000            0.0   

   Bus Line  Bus Station  Bus Stop  Café  Candy Store  Caribbean Restaurant  \
0       0.0     0.000000       0.0  0.00          0.0                   0.0   
1       0.0     0.041667       0.0  0.00          0.0                   0.0   
2       0.0     0.055556       0.0  0.00          0.0                   0.0   
3       0.0     0.000000       0.0  0.02          0.0                   0.0   
4       0.0     0.000000       0.0  0.00          0.0                   0.0   

   Check Cashing Service  Cheese Shop  Chinese Restaurant  Clothing Store  \
0               0.037037         0.00            0.037037             0.0   
1               0.000000         0.00            0.000000             0.0   
2               0.000000         0.00            0.055556             0.0   
3               0.010000         0.01            0.010000             0.0   
4               0.000000         0.00            0.166667             0.0   

   Coffee Shop  Comfort Food Restaurant  Convenience Store  Cosmetics Shop  \
0         0.00                      0.0           0.000000        0.037037   
1         0.00                      0.0           0.041667        0.000000   
2         0.00                      0.0           0.000000        0.000000   
3         0.02                      0.0  

In [47]:
Brooklyn_grouped = Brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
Brooklyn_grouped.head()

Neighborhood  Yoga Studio  ATM  Accessories Store  Airport Terminal  \
0          Bath Beach          0.0  0.0                0.0               0.0   
1           Bay Ridge          0.0  0.0                0.0               0.0   
2  Bedford Stuyvesant          0.0  0.0                0.0               0.0   
3         Bensonhurst          0.0  0.0                0.0               0.0   
4        Bergen Beach          0.0  0.0                0.0               0.0   

   American Restaurant  Antique Shop  Arepa Restaurant  \
0                 0.00           0.0               0.0   
1                 0.04           0.0               0.0   
2                 0.00           0.0               0.0   
3                 0.00           0.0               0.0   
4                 0.00           0.0               0.0   

   Argentinian Restaurant  Art Gallery  Art Museum  Arts & Crafts Store  \
0                     0.0          0.0         0.0                  0.0   
1                     0.0          0.0         0.0                  0.0   
2                     0.0          0.0         0.0                  0.0   
3                     0.0          0.0         0.0                  0.0   
4                     0.0          0.0         0.0                  0.0   

   Arts & Entertainment  Asian Restaurant  Athletics & Sports  Auto Garage  \
0                   0.0          0.018868               0.000          0.0   
1                   0.0          0.000000               0.000          0.0   
2                   0.0          0.000000               0.000          0.0   
3                   0.0          0.000000               0.000          0.0   
4                   0.0          0.000000               0.125          0.0   

   BBQ Joint  Bagel Shop    Bakery      Bank      Bar  Baseball Field  \
0        0.0    0.000000  0.018868  0.018868  0.00000           0.000   
1        0.0    0.040000  0.000000  0.000000  0.04000           0.000   
2        0.0    0.033333  0.000000  0.000000  0.10000           0.000   
3        0.0    0.031250  0.031250  0.031250  0.03125           0.000   
4        0.0    0.000000  0.000000  0.000000  0.00000           0.125   

   Baseball Stadium  Basketball Court  Beach  Beer Bar  Beer Garden  \
0               0.0               0.0    0.0       0.0          0.0   
1               0.0               0.0    0.0       0.0          0.0   
2               0.0               0.0    0.0       0.0          0.0   
3               0.0               0.0    0.0       0.0          0.0   
4               0.0               0.0    0.0       0.0          0.0   

   Beer Store  Big Box Store  Bike Rental / Bike Share  Bike Shop  Bistro  \
0         0.0            0.0                       0.0        0.0     0.0   
1         0.0            0.0                       0.0        0.0     0.0   
2         0.0            0.0                       0.0        0.0     0.0   
3         0.0            0.0                       0.0        0.0     0.0   
4         0.0            0.0                       0.0        0.0     0.0   

   Board Shop  Boat or Ferry  Bookstore  Boutique  Boxing Gym  \
0         0.0            0.0   0.000000  0.000000         0.0   
1         0.0            0.0   0.013333  0.000000         0.0   
2         0.0            0.0   0.000000  0.033333         0.0   
3         0.0            0.0   0.000000  0.000000         0.0   
4         0.0            0.0   0.000000  0.000000         0.0   

   Brazilian Restaurant  Breakfast Spot  Brewery  Bridge  Bubble Tea Shop  \
0                   0.0        0.000000      0.0     0.0         0.056604   
1                   0.0        0.013333      0.0     0.0         0.000000   
2                   0.0        0.000000      0.0     0.0         0.000000   
3                   0.0        0.000000      0.0     0.0         0.000000   
4                   0.0        0.000000      0.0     0.0         0.000000   

   Buffet  Burger Joint  Burmese Restaurant  Burrito Place  Bus Line  \
0   

In [39]:
Queens_grouped = Queens_onehot.groupby('Neighborhood').mean().reset_index()
Queens_grouped.head()

Neighborhood  Yoga Studio  Afghan Restaurant  American Restaurant  \
0          Arverne          0.0                0.0             0.000000   
1          Astoria          0.0                0.0             0.000000   
2  Astoria Heights          0.0                0.0             0.000000   
3       Auburndale          0.0                0.0             0.058824   
4      Bay Terrace          0.0                0.0             0.083333   

   Arepa Restaurant  Argentinian Restaurant  Art Gallery  Art Museum  \
0               0.0                     0.0          0.0         0.0   
1               0.0                     0.0          0.0         0.0   
2               0.0                     0.0          0.0         0.0   
3               0.0                     0.0          0.0         0.0   
4               0.0                     0.0          0.0         0.0   

   Arts & Crafts Store  Arts & Entertainment  Asian Restaurant  \
0                  0.0                   0.0               0.0   
1                  0.0                   0.0               0.0   
2                  0.0                   0.0               0.0   
3                  0.0                   0.0               0.0   
4                  0.0                   0.0               0.0   

   Athletics & Sports  Auto Workshop  Automotive Shop  BBQ Joint  Baby Store  \
0            0.000000            0.0              0.0   0.047619         0.0   
1            0.000000            0.0              0.0   0.010000         0.0   
2            0.000000            0.0              0.0   0.000000         0.0   
3            0.058824            0.0              0.0   0.000000         0.0   
4            0.000000            0.0              0.0   0.000000         0.0   

   Bagel Shop    Bakery      Bank       Bar  Baseball Field  Basketball Court  \
0        0.00  0.000000  0.000000  0.000000             0.0               0.0   
1        0.02  0.030000  0.000000  0.050000             0.0               0.0   
2        0.00  0.083333  0.000000  0.000000             0.0               0.0   
3        0.00  0.000000  0.000000  0.058824             0.0               0.0   
4        0.00  0.027778  0.027778  0.000000             0.0               0.0   

   Bath House     Beach  Beach Bar  Beer Garden  Big Box Store  Bike Trail  \
0         0.0  0.047619        0.0         0.00            0.0         0.0   
1         0.0  0.000000        0.0         0.01            0.0         0.0   
2         0.0  0.000000        0.0         0.00            0.0         0.0   
3         0.0  0.000000        0.0         0.00            0.0         0.0   
4         0.0  0.000000        0.0         0.00            0.0         0.0   

   Bistro  Board Shop  Bookstore  Boutique  Bowling Alley  Boxing Gym  \
0     0.0    0.047619        0.0       0.0       0.000000         0.0   
1     0.0    0.000000        0.0       0.0       0.000000         0.0   
2     0.0    0.000000        0.0       0.0       0.083333         0.0   
3     0.0    0.000000        0.0       0.0       0.000000         0.0   
4     0.0    0.000000        0.0       0.0       0.000000         0.0   

   Brazilian Restaurant  Breakfast Spot  Brewery  Bubble Tea Shop  Buffet  \
0                  0.00             0.0      0.0             0.00     0.0   
1                  0.01             0.0      0.0             0.03     0.0   
2                  0.00             0.0      0.0             0.00     0.0   
3                  0.00             0.0      0.0             0.00     0.0   
4                  0.00             0.0      0.0             0.00     0.0   

   Building  Burger Joint  Burrito Place  Bus Line  Bus Station  Bus Stop  \
0       0.0      0.000000       0.047619       0.0     0.000000  0.047619   
1       0.0      0.010000       0.000000       0.0     0.000000  0.000000   
2       0.0      0.083333       0.000000       0.0     0.083333  0.000000   
3       0.0      0.000000       0.000000       0.0     0.000000  0.000000   
4

In [41]:
Staten_Island_grouped = Staten_Island_onehot.groupby('Neighborhood').mean().reset_index()
Staten_Island_grouped.head()

Neighborhood  ATM  American Restaurant  Antique Shop  Arcade  Art Gallery  \
0       Annadale  0.0             0.090909           0.0     0.0          0.0   
1  Arden Heights  0.0             0.000000           0.0     0.0          0.0   
2      Arlington  0.0             0.166667           0.0     0.0          0.0   
3       Arrochar  0.0             0.000000           0.0     0.0          0.0   
4    Bay Terrace  0.0             0.000000           0.0     0.0          0.0   

   Art Museum  Arts & Crafts Store  Asian Restaurant  Athletics & Sports  \
0         0.0                  0.0               0.0            0.000000   
1         0.0                  0.0               0.0            0.000000   
2         0.0                  0.0               0.0            0.000000   
3         0.0                  0.0               0.0            0.047619   
4         0.0                  0.0               0.0            0.000000   

   BBQ Joint  Bagel Shop    Bakery  Bank       Bar  Baseball Field  \
0        0.0    0.000000  0.090909   0.0  0.090909             0.0   
1        0.0    0.000000  0.000000   0.0  0.000000             0.0   
2        0.0    0.000000  0.000000   0.0  0.000000             0.0   
3        0.0    0.047619  0.000000   0.0  0.000000             0.0   
4        0.0    0.000000  0.000000   0.0  0.000000             0.0   

   Baseball Stadium  Basketball Court  Beach  Beer Bar  Big Box Store  \
0               0.0               0.0    0.0       0.0            0.0   
1               0.0               0.0    0.0       0.0            0.0   
2               0.0               0.0    0.0       0.0            0.0   
3               0.0               0.0    0.0       0.0            0.0   
4               0.0               0.0    0.0       0.0            0.0   

   Board Shop  Boat or Ferry  Bookstore  Bowling Alley  Breakfast Spot  \
0         0.0       0.000000        0.0            0.0             0.0   
1         0.0       0.000000        0.0            0.0             0.0   
2         0.0       0.166667        0.0            0.0             0.0   
3         0.0       0.000000        0.0            0.0             0.0   
4         0.0       0.000000        0.0            0.0             0.0   

   Brewery  Bridal Shop  Bubble Tea Shop  Burger Joint  Bus Station  Bus Stop  \
0      0.0          0.0              0.0           0.0          0.0  0.000000   
1      0.0          0.0              0.0           0.0          0.0  0.166667   
2      0.0          0.0              0.0           0.0          0.0  0.166667   
3      0.0          0.0              0.0           0.0          0.0  0.095238   
4      0.0          0.0              0.0           0.0          0.0  0.000000   

   Business Service  Butcher  Café  Cajun / Creole Restaurant  Campground  \
0               0.0      0.0   0.0                        0.0         0.0   
1               0.0      0.0   0.0                        0.0         0.0   
2               0.0      0.0   0.0                        0.0         0.0   
3               0.0      0.0   0.0                        0.0         0.0   
4               0.0      0.0   0.0                        0.0         0.0   

   Caribbean Restaurant  Check Cashing Service  Chinese Restaurant  \
0                   0.0                    0.0                 0.0   
1                   0.0                    0.0                 0.0   
2                   0.0                    0.0                 0.0   
3                   0.0                    0.0                 0.0   
4                   0.0                    0.0                 0.0   

   Clothing Store  Coffee Shop  Comedy Club  Construction & Landscaping  \
0             0.0     0.000000          0.0                         0.0   
1             0.0     0.166667          0.0                         0.0   
2             0.0     0.166667          0.0                         0.0   
3             0.0     0.000000          0.0                         0.0   
4        

Let's print each neighborhood along with the top 5 most common venues

In [42]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

for hood in Bronx_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Bronx_grouped[Bronx_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

for hood in Brooklyn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Brooklyn_grouped[Brooklyn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

for hood in Queens_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Queens_grouped[Queens_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

for hood in Staten_Island_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Staten_Island_grouped[Staten_Island_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

    

----Battery Park City----
            venue  freq
0            Park  0.09
1     Coffee Shop  0.05
2           Hotel  0.05
3  Clothing Store  0.04
4             Gym  0.03


----Carnegie Hill----
            venue  freq
0     Coffee Shop  0.08
1            Café  0.07
2       Wine Shop  0.04
3     Yoga Studio  0.03
4  Cosmetics Shop  0.03


----Central Harlem----
                 venue  freq
0   African Restaurant  0.07
1           Public Art  0.05
2  American Restaurant  0.05
3                  Bar  0.05
4    French Restaurant  0.05


----Chelsea----
                 venue  freq
0          Coffee Shop  0.06
1               Bakery  0.05
2  American Restaurant  0.04
3          Art Gallery  0.04
4                Hotel  0.03


----Chinatown----
                 venue  freq
0   Chinese Restaurant  0.08
1               Bakery  0.08
2         Cocktail Bar  0.05
3         Dessert Shop  0.04
4  American Restaurant  0.04


----Civic Center----
                  venue  freq
0           Coffee Shop 



----Fieldston----
                 venue  freq
0          Music Venue  0.33
1                River  0.33
2                Plaza  0.33
3  Peruvian Restaurant  0.00
4            Nightclub  0.00


----Fordham----
                venue  freq
0      Clothing Store  0.06
1          Shoe Store  0.06
2          Donut Shop  0.06
3   Mobile Phone Shop  0.06
4  Spanish Restaurant  0.05


----High  Bridge----
                       venue  freq
0                   Pharmacy  0.13
1                Pizza Place  0.10
2         Seafood Restaurant  0.10
3         Chinese Restaurant  0.10
4  Latin American Restaurant  0.07


----Hunts Point----
                venue  freq
0           Juice Bar  0.08
1  Seafood Restaurant  0.08
2        Gourmet Shop  0.08
3                Food  0.08
4       Grocery Store  0.08


----Kingsbridge----
                       venue  freq
0                Pizza Place  0.09
1                        Bar  0.06
2             Sandwich Place  0.06
3  Latin American Restaurant  0.04


NameError: name 'Brooklyn_grouped' is not defined

let us load everything to daatframe

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [55]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_manhattan = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_manhattan['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted_manhattan.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_manhattan.head()

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
0  Battery Park City                  Park           Coffee Shop   
1      Carnegie Hill           Coffee Shop                  Café   
2     Central Harlem    African Restaurant            Public Art   
3            Chelsea           Coffee Shop                Bakery   
4          Chinatown    Chinese Restaurant                Bakery   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0                 Hotel        Clothing Store                   Gym   
1             Wine Shop           Yoga Studio        Cosmetics Shop   
2   American Restaurant                   Bar     French Restaurant   
3   American Restaurant           Art Gallery                 Hotel   
4          Cocktail Bar          Dessert Shop   American Restaurant   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0            Playground         Memorial Site             BBQ Joint   
1                   Bar                   Gym             Bookstore   
2    Seafood Restaurant  Gym / Fitness Center    Chinese Restaurant   
3             Wine Shop     French Restaurant        Ice Cream Shop   
4          Optical Shop    Salon / Barbershop     Hotpot Restaurant   

  9th Most Common Venue 10th Most Common Venue  
0           Pizza Place             Food Court  
1     French Restaurant   Gym / Fitness Center  
2                  Park            Music Venue  
3               Theater           Dessert Shop  
4       Bubble Tea Shop  Vietnamese Restaurant

In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_Bronx = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_Bronx['Neighborhood'] = Bronx_grouped['Neighborhood']

for ind in np.arange(Bronx_grouped.shape[0]):
    neighborhoods_venues_sorted_Bronx.iloc[ind, 1:] = return_most_common_venues(Bronx_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_Bronx.head()

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
0      Allerton           Pizza Place        Discount Store   
1    Baychester            Donut Shop        Baseball Field   
2  Bedford Park                 Diner         Deli / Bodega   
3       Belmont    Italian Restaurant           Pizza Place   
4     Bronxdale    Chinese Restaurant    Italian Restaurant   

         3rd Most Common Venue 4th Most Common Venue  \
0                Deli / Bodega           Supermarket   
1                         Bank           Bus Station   
2                  Pizza Place    Mexican Restaurant   
3                Deli / Bodega                Bakery   
4  Eastern European Restaurant    Spanish Restaurant   

           5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0               Department Store    Chinese Restaurant           Gas Station   
1             Spanish Restaurant          Burger Joint   Sporting Goods Shop   
2                    Bus Station   Fried Chicken Joint            Food Truck   
3             Spanish Restaurant          Dessert Shop         Grocery Store   
4  Paper / Office Supplies Store    Mexican Restaurant                   Gym   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0                    Spa            Smoke Shop         Breakfast Spot  
1              Pet Store         Shopping Mall            Supermarket  
2     Chinese Restaurant        Sandwich Place            Supermarket  
3                   Bank           Coffee Shop                    Bar  
4  Performing Arts Venue                  Bank         Breakfast Spot

In [48]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_Brooklyn = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_Brooklyn['Neighborhood'] = Brooklyn_grouped['Neighborhood']

for ind in np.arange(Brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted_Brooklyn.iloc[ind, 1:] = return_most_common_venues(Brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_Brooklyn.head()

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
0          Bath Beach           Pizza Place       Bubble Tea Shop   
1           Bay Ridge                   Spa    Italian Restaurant   
2  Bedford Stuyvesant                   Bar           Coffee Shop   
3         Bensonhurst    Chinese Restaurant    Italian Restaurant   
4        Bergen Beach                  Park       Harbor / Marina   

  3rd Most Common Venue   4th Most Common Venue 5th Most Common Venue  \
0    Chinese Restaurant    Fast Food Restaurant           Gas Station   
1           Pizza Place                     Bar      Greek Restaurant   
2                  Café  Thrift / Vintage Store              Boutique   
3            Donut Shop        Sushi Restaurant        Ice Cream Shop   
4        Baseball Field           Moving Target            Playground   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0              Pharmacy      Sushi Restaurant    Italian Restaurant   
1   American Restaurant            Bagel Shop            Playground   
2           Bus Station              Bus Stop          Cocktail Bar   
3          Dessert Shop          Liquor Store        Cosmetics Shop   
4    Athletics & Sports           Yoga Studio       Organic Grocery   

  9th Most Common Venue 10th Most Common Venue  
0            Donut Shop             Sports Bar  
1              Pharmacy         Sandwich Place  
2      Community Center     Italian Restaurant  
3                   Bar                   Bank  
4     Other Repair Shop        Other Nightlife

In [49]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_Queens = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_Queens['Neighborhood'] = Queens_grouped['Neighborhood']

for ind in np.arange(Queens_grouped.shape[0]):
    neighborhoods_venues_sorted_Queens.iloc[ind, 1:] = return_most_common_venues(Queens_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_Queens.head()

Neighborhood 1st Most Common Venue      2nd Most Common Venue  \
0          Arverne             Surf Spot              Metro Station   
1          Astoria      Greek Restaurant  Middle Eastern Restaurant   
2  Astoria Heights         Deli / Bodega                Pizza Place   
3       Auburndale    Italian Restaurant                  Pet Store   
4      Bay Terrace        Clothing Store        American Restaurant   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0        Sandwich Place              Bus Stop           Coffee Shop   
1                   Bar                  Café    Seafood Restaurant   
2           Bus Station           Supermarket         Bowling Alley   
3           Supermarket  Fast Food Restaurant                   Bar   
4         Women's Store        Cosmetics Shop            Shoe Store   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0                 Beach            Board Shop              Tiki Bar   
1            Hookah Bar                Bakery       Bubble Tea Shop   
2          Burger Joint                Bakery                Hostel   
3      Toy / Game Store            Hookah Bar    Miscellaneous Shop   
4     Mobile Phone Shop            Kids Store            Donut Shop   

  9th Most Common Venue 10th Most Common Venue  
0             BBQ Joint      French Restaurant  
1     Indian Restaurant            Pizza Place  
2                Museum                  Plaza  
3     Mobile Phone Shop         Gymnastics Gym  
4           Pizza Place               Pharmacy

In [50]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_Staten_Island = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_Staten_Island['Neighborhood'] = Staten_Island_grouped['Neighborhood']

for ind in np.arange(Staten_Island_grouped.shape[0]):
    neighborhoods_venues_sorted_Staten_Island.iloc[ind, 1:] = return_most_common_venues(Staten_Island_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_Staten_Island.head()

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
0       Annadale           Pizza Place                   Bar   
1  Arden Heights         Deli / Bodega            Playground   
2      Arlington         Deli / Bodega   American Restaurant   
3       Arrochar         Deli / Bodega    Italian Restaurant   
4    Bay Terrace           Supermarket    Salon / Barbershop   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0                 Diner   American Restaurant          Dance Studio   
1              Bus Stop           Coffee Shop              Pharmacy   
2         Boat or Ferry              Bus Stop           Coffee Shop   
3              Bus Stop           Pizza Place                 Hotel   
4      Sushi Restaurant        Shipping Store          Home Service   

  6th Most Common Venue  7th Most Common Venue  8th Most Common Venue  \
0          Liquor Store             Restaurant          Train Station   
1           Pizza Place  Outdoors & Recreation            Outlet Mall   
2         Grocery Store                   Pier  Outdoors & Recreation   
3           Supermarket             Food Truck               Pharmacy   
4         Train Station     Italian Restaurant             Donut Shop   

  9th Most Common Venue     10th Most Common Venue  
0                Bakery                   Pharmacy  
1                  Park        Peruvian Restaurant  
2           Outlet Mall                       Park  
3     Polish Restaurant  Middle Eastern Restaurant  
4      Insurance Office        Peruvian Restaurant

## Cluster Neighborhoods for each Borough

Run k-means to cluster the neighborhood into 5 clusters.

In [51]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans_manhattan = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans_manhattan.labels_[0:10] )

Bronx_grouped_clustering = Bronx_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans_Bronx = KMeans(n_clusters=kclusters, random_state=0).fit(Bronx_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans_Bronx.labels_[0:10] )
Brooklyn_grouped_clustering = Brooklyn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans_Brooklyn = KMeans(n_clusters=kclusters, random_state=0).fit(Brooklyn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans_Brooklyn.labels_[0:10] )

Queens_grouped_clustering = Queens_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans_Queens = KMeans(n_clusters=kclusters, random_state=0).fit(Queens_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans_Queens.labels_[0:10] )
Staten_Island_grouped_clustering = Staten_Island_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans_Staten_Island = KMeans(n_clusters=kclusters, random_state=0).fit(Staten_Island_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans_Staten_Island.labels_[0:10] )

[3 0 1 0 0 1 1 3 0 1]
[0 0 0 4 0 0 0 3 0 3]
[2 2 2 2 1 2 2 2 2 2]
[4 4 4 4 4 4 3 4 4 4]
[1 1 3 1 1 3 1 1 1 1]


Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [52]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans_manhattan.labels_)

manhattan_merged = manhattan_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()# check the last columns!

Borough        Neighborhood   Latitude  Longitude  Cluster Labels  \
0  Manhattan         Marble Hill  40.876551 -73.910660               1   
1  Manhattan           Chinatown  40.715618 -73.994279               0   
2  Manhattan  Washington Heights  40.851903 -73.936900               0   
3  Manhattan              Inwood  40.867684 -73.921210               3   
4  Manhattan    Hamilton Heights  40.823604 -73.949688               3   

  1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
0                   Gym           Coffee Shop        Sandwich Place   
1    Chinese Restaurant                Bakery          Cocktail Bar   
2                  Café                Bakery         Grocery Store   
3    Mexican Restaurant            Restaurant                  Café   
4           Pizza Place                  Café           Coffee Shop   

  4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
0           Yoga Studio      Department Store            Steakhouse   
1          Dessert Shop   American Restaurant          Optical Shop   
2    Spanish Restaurant    Chinese Restaurant     Mobile Phone Shop   
3                Bakery  Caribbean Restaurant    Chinese Restaurant   
4         Deli / Bodega    Mexican Restaurant           Yoga Studio   

  7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
0        Clothing Store    Seafood Restaurant            Kids Store   
1    Salon / Barbershop     Hotpot Restaurant       Bubble Tea Shop   
2                  Bank           Pizza Place        Sandwich Place   
3                 Diner           Pizza Place                  Park   
4                  Park                School        Sandwich Place   

  10th Most Common Venue  
0                  Diner  
1  Vietnamese Restaurant  
2       Tapas Restaurant  
3                 Lounge  
4     Chinese Restaurant

In [56]:
# add clustering labels
neighborhoods_venues_sorted_manhattan.insert(0, 'Cluster Labels', kmeans_manhattan.labels_)

manhattan_merged = manhattan_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()# check the last columns!

Borough        Neighborhood   Latitude  Longitude  Cluster Labels  \
0  Manhattan         Marble Hill  40.876551 -73.910660               1   
1  Manhattan           Chinatown  40.715618 -73.994279               0   
2  Manhattan  Washington Heights  40.851903 -73.936900               0   
3  Manhattan              Inwood  40.867684 -73.921210               3   
4  Manhattan    Hamilton Heights  40.823604 -73.949688               3   

  1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
0                   Gym           Coffee Shop        Sandwich Place   
1    Chinese Restaurant                Bakery          Cocktail Bar   
2                  Café                Bakery         Grocery Store   
3    Mexican Restaurant            Restaurant                  Café   
4           Pizza Place                  Café           Coffee Shop   

  4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
0           Yoga Studio      Department Store            Steakhouse   
1          Dessert Shop   American Restaurant          Optical Shop   
2    Spanish Restaurant    Chinese Restaurant     Mobile Phone Shop   
3                Bakery  Caribbean Restaurant    Chinese Restaurant   
4         Deli / Bodega    Mexican Restaurant           Yoga Studio   

  7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
0        Clothing Store    Seafood Restaurant            Kids Store   
1    Salon / Barbershop     Hotpot Restaurant       Bubble Tea Shop   
2                  Bank           Pizza Place        Sandwich Place   
3                 Diner           Pizza Place                  Park   
4                  Park                School        Sandwich Place   

  10th Most Common Venue  
0                  Diner  
1  Vietnamese Restaurant  
2       Tapas Restaurant  
3                 Lounge  
4     Chinese Restaurant

In [63]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_Bronx = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_Bronx['Neighborhood'] = Bronx_grouped['Neighborhood']

for ind in np.arange(Bronx_grouped.shape[0]):
    neighborhoods_venues_sorted_Bronx.iloc[ind, 1:] = return_most_common_venues(Bronx_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_Bronx.head()
# add clustering labels
neighborhoods_venues_sorted_Bronx.insert(0, 'Cluster Labels', kmeans_Bronx.labels_)

Bronx_merged = Bronx_data

# merge Bronx_grouped with Bronx_data to add latitude/longitude for each neighborhood
Bronx_merged = Bronx_merged.join(neighborhoods_venues_sorted_Bronx.set_index('Neighborhood'), on='Neighborhood')

Bronx_merged.head()# check the last columns!

Borough Neighborhood   Latitude  Longitude  Cluster Labels  \
0   Bronx    Wakefield  40.894705 -73.847201               0   
1   Bronx   Co-op City  40.874294 -73.829939               3   
2   Bronx  Eastchester  40.887556 -73.827806               0   
3   Bronx    Fieldston  40.895437 -73.905643               1   
4   Bronx    Riverdale  40.890834 -73.912585               0   

  1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
0              Pharmacy            Donut Shop            Laundromat   
1            Bagel Shop            Donut Shop           Bus Station   
2  Caribbean Restaurant         Deli / Bodega           Bus Station   
3           Music Venue                 River                 Plaza   
4                  Park           Bus Station                 Plaza   

  4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
0  Caribbean Restaurant        Sandwich Place         Deli / Bodega   
1         Grocery Store              Pharmacy  Fast Food Restaurant   
2                 Diner  Fast Food Restaurant              Bus Stop   
3   Peruvian Restaurant             Nightclub                Office   
4  Medical Supply Store                   Gym                  Bank   

   7th Most Common Venue 8th Most Common Venue          9th Most Common Venue  \
0           Dessert Shop        Ice Cream Shop                          Plaza   
1            Pizza Place    Chinese Restaurant                           Park   
2          Bowling Alley     Food & Drink Shop                    Pizza Place   
3  Outdoors & Recreation          Outlet Store  Paper / Office Supplies Store   
4      Accessories Store   Peruvian Restaurant          Outdoors & Recreation   

  10th Most Common Venue  
0           Outlet Store  
1     Salon / Barbershop  
2               Platform  
3                   Park  
4           Outlet Store

In [64]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_Brooklyn = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_Brooklyn['Neighborhood'] = Brooklyn_grouped['Neighborhood']

for ind in np.arange(Brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted_Brooklyn.iloc[ind, 1:] = return_most_common_venues(Brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_Brooklyn.head()
# add clustering labels
neighborhoods_venues_sorted_Brooklyn.insert(0, 'Cluster Labels', kmeans_Brooklyn.labels_)

Brooklyn_merged = Brooklyn_data

# merge Brooklyn_grouped with Brooklyn_data to add latitude/longitude for each neighborhood
Brooklyn_merged = Brooklyn_merged.join(neighborhoods_venues_sorted_Brooklyn.set_index('Neighborhood'), on='Neighborhood')

Brooklyn_merged.head()# check the last columns!

Borough Neighborhood   Latitude  Longitude  Cluster Labels  \
0  Brooklyn    Bay Ridge  40.625801 -74.030621               2   
1  Brooklyn  Bensonhurst  40.611009 -73.995180               2   
2  Brooklyn  Sunset Park  40.645103 -74.010316               2   
3  Brooklyn   Greenpoint  40.730201 -73.954241               2   
4  Brooklyn    Gravesend  40.595260 -73.973471               2   

  1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
0                   Spa    Italian Restaurant           Pizza Place   
1    Chinese Restaurant    Italian Restaurant            Donut Shop   
2    Mexican Restaurant                  Bank           Pizza Place   
3           Coffee Shop                   Bar           Pizza Place   
4                Bakery           Pizza Place    Italian Restaurant   

  4th Most Common Venue      5th Most Common Venue 6th Most Common Venue  \
0                   Bar           Greek Restaurant   American Restaurant   
1      Sushi Restaurant             Ice Cream Shop          Dessert Shop   
2                Bakery  Latin American Restaurant   Fried Chicken Joint   
3          Cocktail Bar                Record Shop     French Restaurant   
4           Bus Station                        Bar                Lounge   

  7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
0            Bagel Shop            Playground              Pharmacy   
1          Liquor Store        Cosmetics Shop                   Bar   
2                   Gym     Mobile Phone Shop              Pharmacy   
3           Yoga Studio    Mexican Restaurant         Garden Center   
4                   Spa        Breakfast Spot         Metro Station   

  10th Most Common Venue  
0         Sandwich Place  
1                   Bank  
2            Record Shop  
3               Boutique  
4         Farmers Market

In [59]:
neighborhoods_venues_sorted_Brooklyn.head()

Cluster Labels        Neighborhood 1st Most Common Venue  \
0               2          Bath Beach           Pizza Place   
1               2           Bay Ridge                   Spa   
2               2  Bedford Stuyvesant                   Bar   
3               2         Bensonhurst    Chinese Restaurant   
4               1        Bergen Beach                  Park   

  2nd Most Common Venue 3rd Most Common Venue   4th Most Common Venue  \
0       Bubble Tea Shop    Chinese Restaurant    Fast Food Restaurant   
1    Italian Restaurant           Pizza Place                     Bar   
2           Coffee Shop                  Café  Thrift / Vintage Store   
3    Italian Restaurant            Donut Shop        Sushi Restaurant   
4       Harbor / Marina        Baseball Field           Moving Target   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0           Gas Station              Pharmacy      Sushi Restaurant   
1      Greek Restaurant   American Restaurant            Bagel Shop   
2              Boutique           Bus Station              Bus Stop   
3        Ice Cream Shop          Dessert Shop          Liquor Store   
4            Playground    Athletics & Sports           Yoga Studio   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0    Italian Restaurant            Donut Shop             Sports Bar  
1            Playground              Pharmacy         Sandwich Place  
2          Cocktail Bar      Community Center     Italian Restaurant  
3        Cosmetics Shop                   Bar                   Bank  
4       Organic Grocery     Other Repair Shop        Other Nightlife

In [74]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_Queens = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_Queens['Neighborhood'] = Queens_grouped['Neighborhood']

for ind in np.arange(Queens_grouped.shape[0]):
    neighborhoods_venues_sorted_Queens.iloc[ind, 1:] = return_most_common_venues(Queens_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_Queens.head()
# add clustering labels
neighborhoods_venues_sorted_Queens.insert(0, 'Cluster Labels', kmeans_Queens.labels_)

Queens_merged = Queens_data

# merge Queens_grouped with Queens_data to add latitude/longitude for each neighborhood
Queens_merged = Queens_merged.join(neighborhoods_venues_sorted_Queens.set_index('Neighborhood'), on='Neighborhood')

Queens_merged.head()# check the last columns!

Borough     Neighborhood   Latitude  Longitude  Cluster Labels  \
0  Queens          Astoria  40.768509 -73.915654               4   
1  Queens         Woodside  40.746349 -73.901842               4   
2  Queens  Jackson Heights  40.751981 -73.882821               4   
3  Queens         Elmhurst  40.744049 -73.881656               4   
4  Queens     Howard Beach  40.654225 -73.838138               4   

       1st Most Common Venue      2nd Most Common Venue  \
0           Greek Restaurant  Middle Eastern Restaurant   
1              Grocery Store        Filipino Restaurant   
2  Latin American Restaurant        Peruvian Restaurant   
3            Thai Restaurant         Mexican Restaurant   
4         Italian Restaurant                   Pharmacy   

       3rd Most Common Venue  4th Most Common Venue 5th Most Common Venue  \
0                        Bar                   Café    Seafood Restaurant   
1                     Bakery        Thai Restaurant                   Bar   
2  South American Restaurant                 Bakery     Mobile Phone Shop   
3      Vietnamese Restaurant  Indonesian Restaurant       Bubble Tea Shop   
4             Sandwich Place   Fast Food Restaurant            Hobby Shop   

       6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0                 Hookah Bar                Bakery       Bubble Tea Shop   
1  Latin American Restaurant            Donut Shop                   Pub   
2            Thai Restaurant         Grocery Store           Supermarket   
3         Chinese Restaurant            Food Court      Sushi Restaurant   
4                      Motel           Supermarket  Gym / Fitness Center   

  9th Most Common Venue 10th Most Common Venue  
0     Indian Restaurant            Pizza Place  
1   American Restaurant            Pizza Place  
2    Spanish Restaurant            Pizza Place  
3  Gym / Fitness Center                   Bank  
4      Sushi Restaurant                    Gym

In [78]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_Staten_Island = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_Staten_Island['Neighborhood'] = Staten_Island_grouped['Neighborhood']

for ind in np.arange(Staten_Island_grouped.shape[0]):
    neighborhoods_venues_sorted_Staten_Island.iloc[ind, 1:] = return_most_common_venues(Staten_Island_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_Staten_Island.head()
# add clustering labels
neighborhoods_venues_sorted_Staten_Island.insert(0, 'Cluster Labels', kmeans_Staten_Island.labels_)

Staten_Island_merged = Staten_Island_data

# merge Staten_Island_grouped with Staten_Island_data to add latitude/longitude for each neighborhood
Staten_Island_merged = Staten_Island_merged.join(neighborhoods_venues_sorted_Staten_Island.set_index('Neighborhood'), on='Neighborhood')

Staten_Island_merged.head()# check the last columns!

Borough   Neighborhood   Latitude  Longitude  Cluster Labels  \
0  Staten Island     St. George  40.644982 -74.079353             1.0   
1  Staten Island   New Brighton  40.640615 -74.087017             3.0   
2  Staten Island      Stapleton  40.626928 -74.077902             1.0   
3  Staten Island       Rosebank  40.615305 -74.069805             1.0   
4  Staten Island  West Brighton  40.631879 -74.107182             1.0   

  1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
0        Clothing Store           Pizza Place   Sporting Goods Shop   
1         Deli / Bodega              Bus Stop                  Park   
2        Discount Store        Sandwich Place                   Bar   
3    Italian Restaurant         Grocery Store          Intersection   
4           Coffee Shop        Cosmetics Shop           Music Store   

  4th Most Common Venue      5th Most Common Venue 6th Most Common Venue  \
0    Italian Restaurant        American Restaurant                   Bar   
1           Bridal Shop             Discount Store            Playground   
2                  Bank                Pizza Place        Cosmetics Shop   
3   Filipino Restaurant  Cajun / Creole Restaurant        Cosmetics Shop   
4    Italian Restaurant                        Bar                  Bank   

  7th Most Common Venue 8th Most Common Venue        9th Most Common Venue  \
0            Donut Shop                  Park                 Burger Joint   
1         Bowling Alley                Market          Martial Arts School   
2       Motorcycle Shop      Department Store                   Skate Park   
3        Discount Store            Restaurant  Eastern European Restaurant   
4        Breakfast Spot         Deli / Bodega                   Board Shop   

  10th Most Common Venue  
0            Coffee Shop  
1           Liquor Store  
2   Fast Food Restaurant  
3               Pharmacy  
4                  Diner

Finally, let's visualize the resulting clusters for each Borough

In [67]:
# create map
map_clusters_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_manhattan)
       
map_clusters_manhattan

In [68]:
# create map
map_clusters_Bronx = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Bronx_merged['Latitude'], Bronx_merged['Longitude'], Bronx_merged['Neighborhood'], Bronx_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_Bronx)
       
map_clusters_Bronx

In [69]:
# create map
map_clusters_Brooklyn = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Brooklyn_merged['Latitude'], Brooklyn_merged['Longitude'], Brooklyn_merged['Neighborhood'], Brooklyn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_Brooklyn)
       
map_clusters_Brooklyn

In [70]:
# create map
map_clusters_Queens = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Queens_merged['Latitude'], Queens_merged['Longitude'], Queens_merged['Neighborhood'], Queens_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_Queens)
       
map_clusters_Queens

In [80]:
# create map
map_clusters_Staten_Island = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Staten_Island_merged['Latitude'], Staten_Island_merged['Longitude'], Staten_Island_merged['Neighborhood'], Staten_Island_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_Staten_Island)
       
map_clusters_Staten_Island

TypeError: list indices must be integers or slices, not float

### Examine Clusters

In [82]:
for
manhattan_merged.loc[manhattan_merged['Cluster Labels'] ==0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
1            Chinatown    Chinese Restaurant                Bakery   
2   Washington Heights                  Café                Bakery   
5       Manhattanville           Coffee Shop                   Bar   
9            Yorkville    Italian Restaurant           Coffee Shop   
10          Lenox Hill    Italian Restaurant           Coffee Shop   
12     Upper West Side    Italian Restaurant                Bakery   
17             Chelsea           Coffee Shop                Bakery   
18   Greenwich Village    Italian Restaurant      Sushi Restaurant   
19        East Village                   Bar           Pizza Place   
20     Lower East Side    Chinese Restaurant           Pizza Place   
22        Little Italy                  Café                Bakery   
23                Soho        Clothing Store              Boutique   
25    Manhattan Valley           Coffee Shop    Mexican Restaurant   
27            Gramercy                   Bar    Italian Restaurant   
30       Carnegie Hill           Coffee Shop                  Café   
31                Noho          Cocktail Bar    Italian Restaurant   
34        Sutton Place    Italian Restaurant           Pizza Place   
35          Turtle Bay    Italian Restaurant           Coffee Shop   

   3rd Most Common Venue 4th Most Common Venue     5th Most Common Venue  \
1           Cocktail Bar          Dessert Shop       American Restaurant   
2          Grocery Store    Spanish Restaurant        Chinese Restaurant   
5          Deli / Bodega    Seafood Restaurant        Italian Restaurant   
9                    Gym                   Bar          Sushi Restaurant   
10      Sushi Restaurant          Cocktail Bar               Pizza Place   
12                   Bar                  Café                Bagel Shop   
17   American Restaurant           Art Gallery                     Hotel   
18        Clothing Store              Boutique            Cosmetics Shop   
19    Mexican Restaurant             Speakeasy     Vietnamese Restaurant   
20  Caribbean Restaurant                Bakery               Art Gallery   
22    Italian Restaurant    Chinese Restaurant            Ice Cream Shop   
23    Italian Restaurant            Shoe Store  Mediterranean Restaurant   
25                   Bar            Playground               Pizza Place   
27            Bagel Shop           Pizza Place                Playground   
30             Wine Shop           Yoga Studio            Cosmetics Shop   
31                 Hotel     French Restaurant               Pizza Place   
34  Gym / Fitness Center     Indian Restaurant                      Park   
35      Sushi Restaurant                 Hotel       Japanese Restaurant   

     6th Most Common Venue      7th Most Common Venue 8th Most Common Venue  \
1             Optical Shop         Salon / Barbershop     Hotpot Restaurant   
2        Mobile Phone Shop                       Bank           Pizza Place   
5       Chinese Restaurant         Mexican Restaurant                  Park   
9            Deli / Bodega        Japanese Restaurant             Wine Shop   
10            Burger Joint                       Café                   Gym   
12             Coffee Shop                   Wine Bar       Thai Restaurant   
17               Wine Shop          French Restaurant        Ice Cream Shop   
18                  Bakery               Dessert Shop           Coffee Shop   
19            Cocktail Bar             Ice Cream Shop              Wine Bar   
20                    Café  Latin American Restaurant      Ramen Restaurant   
22            Cocktail Bar                Pizza Place        Sandwich Place   
23           Women's Store                      Hotel           Coffee Shop   
25   Vietnamese Restaurant                Yoga Studio                Hostel   
27                    Café                  Wine Shop   American Restaurant   
30                     Bar                       

In [87]:
idx=[0,1,2,3,4,5]
for i in idx:
    display(manhattan_merged.loc[manhattan_merged['Cluster Labels'] ==i, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]])

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
1            Chinatown    Chinese Restaurant                Bakery   
2   Washington Heights                  Café                Bakery   
5       Manhattanville           Coffee Shop                   Bar   
9            Yorkville    Italian Restaurant           Coffee Shop   
10          Lenox Hill    Italian Restaurant           Coffee Shop   
12     Upper West Side    Italian Restaurant                Bakery   
17             Chelsea           Coffee Shop                Bakery   
18   Greenwich Village    Italian Restaurant      Sushi Restaurant   
19        East Village                   Bar           Pizza Place   
20     Lower East Side    Chinese Restaurant           Pizza Place   
22        Little Italy                  Café                Bakery   
23                Soho        Clothing Store              Boutique   
25    Manhattan Valley           Coffee Shop    Mexican Restaurant   
27            Gramercy                   Bar    Italian Restaurant   
30       Carnegie Hill           Coffee Shop                  Café   
31                Noho          Cocktail Bar    Italian Restaurant   
34        Sutton Place    Italian Restaurant           Pizza Place   
35          Turtle Bay    Italian Restaurant           Coffee Shop   

   3rd Most Common Venue 4th Most Common Venue     5th Most Common Venue  \
1           Cocktail Bar          Dessert Shop       American Restaurant   
2          Grocery Store    Spanish Restaurant        Chinese Restaurant   
5          Deli / Bodega    Seafood Restaurant        Italian Restaurant   
9                    Gym                   Bar          Sushi Restaurant   
10      Sushi Restaurant          Cocktail Bar               Pizza Place   
12                   Bar                  Café                Bagel Shop   
17   American Restaurant           Art Gallery                     Hotel   
18        Clothing Store              Boutique            Cosmetics Shop   
19    Mexican Restaurant             Speakeasy     Vietnamese Restaurant   
20  Caribbean Restaurant                Bakery               Art Gallery   
22    Italian Restaurant    Chinese Restaurant            Ice Cream Shop   
23    Italian Restaurant            Shoe Store  Mediterranean Restaurant   
25                   Bar            Playground               Pizza Place   
27            Bagel Shop           Pizza Place                Playground   
30             Wine Shop           Yoga Studio            Cosmetics Shop   
31                 Hotel     French Restaurant               Pizza Place   
34  Gym / Fitness Center     Indian Restaurant                      Park   
35      Sushi Restaurant                 Hotel       Japanese Restaurant   

     6th Most Common Venue      7th Most Common Venue 8th Most Common Venue  \
1             Optical Shop         Salon / Barbershop     Hotpot Restaurant   
2        Mobile Phone Shop                       Bank           Pizza Place   
5       Chinese Restaurant         Mexican Restaurant                  Park   
9            Deli / Bodega        Japanese Restaurant             Wine Shop   
10            Burger Joint                       Café                   Gym   
12             Coffee Shop                   Wine Bar       Thai Restaurant   
17               Wine Shop          French Restaurant        Ice Cream Shop   
18                  Bakery               Dessert Shop           Coffee Shop   
19            Cocktail Bar             Ice Cream Shop              Wine Bar   
20                    Café  Latin American Restaurant      Ramen Restaurant   
22            Cocktail Bar                Pizza Place        Sandwich Place   
23           Women's Store                      Hotel           Coffee Shop   
25   Vietnamese Restaurant                Yoga Studio                Hostel   
27                    Café                  Wine Shop   American Restaurant   
30                     Bar                       

Neighborhood 1st Most Common Venue    2nd Most Common Venue  \
0          Marble Hill                   Gym              Coffee Shop   
6       Central Harlem    African Restaurant               Public Art   
8      Upper East Side    Italian Restaurant                  Exhibit   
13      Lincoln Square                 Plaza     Gym / Fitness Center   
14             Clinton    Italian Restaurant                  Theater   
15             Midtown                 Hotel      Sporting Goods Shop   
16         Murray Hill                 Hotel           Sandwich Place   
21             Tribeca   American Restaurant       Italian Restaurant   
24        West Village    Italian Restaurant  New American Restaurant   
29  Financial District           Coffee Shop              Pizza Place   
32        Civic Center           Coffee Shop                      Spa   
38            Flatiron    Italian Restaurant      American Restaurant   
39        Hudson Yards  Gym / Fitness Center      American Restaurant   

   3rd Most Common Venue 4th Most Common Venue    5th Most Common Venue  \
0         Sandwich Place           Yoga Studio         Department Store   
6    American Restaurant                   Bar        French Restaurant   
8                 Bakery           Coffee Shop     Gym / Fitness Center   
13               Theater                  Café             Concert Hall   
14  Gym / Fitness Center           Coffee Shop                      Gym   
15           Coffee Shop        Clothing Store                  Theater   
16           Coffee Shop   Japanese Restaurant      American Restaurant   
21                  Park                  Café                 Wine Bar   
24   American Restaurant          Cocktail Bar                     Park   
29          Cocktail Bar   American Restaurant                     Park   
32  Gym / Fitness Center   American Restaurant                     Park   
38                   Spa   Japanese Restaurant  New American Restaurant   
39                 Hotel            Restaurant                     Café   

    6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0              Steakhouse        Clothing Store    Seafood Restaurant   
6      Seafood Restaurant  Gym / Fitness Center    Chinese Restaurant   
8               Juice Bar                   Spa                 Hotel   
13  Performing Arts Venue                  Park             Wine Shop   
14         Sandwich Place   American Restaurant                   Spa   
15                    Spa             Bookstore   American Restaurant   
16   Gym / Fitness Center                   Bar            Taco Place   
21                    Spa      Greek Restaurant  Gym / Fitness Center   
24               Wine Bar           Coffee Shop        Cosmetics Shop   
29                    Bar    Italian Restaurant                   Gym   
32     Italian Restaurant          Cocktail Bar     French Restaurant   
38            Yoga Studio   Sporting Goods Shop             Wine Shop   
39     Italian Restaurant           Coffee Shop       Thai Restaurant   

       9th Most Common Venue 10th Most Common Venue  
0                 Kids Store                  Diner  
6                       Park            Music Venue  
8             Cosmetics Shop            Yoga Studio  
13                Food Truck      French Restaurant  
14                 Wine Shop           Cocktail Bar  
15                Steakhouse      Indian Restaurant  
16        Chinese Restaurant                    Spa  
21                Steakhouse            Coffee Shop  
24                 Jazz Club         Ice Cream Shop  
29                Steakhouse            Event Space  
32                     Hotel           Dance Studio  
38  Mediterranean Restaurant            Coffee Shop  
39                   Dog Run                   Park

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
37  Stuyvesant Town                   Bar                  Park   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
37           Coffee Shop       Harbor / Marina          Cocktail Bar   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
37        Farmers Market              Heliport           Pet Service   

   9th Most Common Venue 10th Most Common Venue  
37                Bistro          Boat or Ferry

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
3                Inwood    Mexican Restaurant            Restaurant   
4      Hamilton Heights           Pizza Place                  Café   
7           East Harlem    Mexican Restaurant         Deli / Bodega   
11     Roosevelt Island           Coffee Shop                  Park   
26  Morningside Heights           Coffee Shop   American Restaurant   
28    Battery Park City                  Park           Coffee Shop   
36           Tudor City    Mexican Restaurant                  Park   

   3rd Most Common Venue 4th Most Common Venue      5th Most Common Venue  \
3                   Café                Bakery       Caribbean Restaurant   
4            Coffee Shop         Deli / Bodega         Mexican Restaurant   
7        Thai Restaurant                Bakery  Latin American Restaurant   
11         Metro Station  Gym / Fitness Center                     Bridge   
26                  Park             Bookstore               Burger Joint   
28                 Hotel        Clothing Store                        Gym   
36                  Café                 Diner                Pizza Place   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
3     Chinese Restaurant                 Diner           Pizza Place   
4            Yoga Studio                  Park                School   
7         Sandwich Place                   Spa            Donut Shop   
11       Bubble Tea Shop           Supermarket                   Gym   
26        Sandwich Place                  Café         Deli / Bodega   
28            Playground         Memorial Site             BBQ Joint   
36           Coffee Shop      Greek Restaurant                   Gym   

   9th Most Common Venue 10th Most Common Venue  
3                   Park                 Lounge  
4         Sandwich Place     Chinese Restaurant  
7            Gas Station             Taco Place  
11      Greek Restaurant  Outdoors & Recreation  
26  Ethiopian Restaurant            Gaming Cafe  
28           Pizza Place             Food Court  
36               Dog Run                 Garden

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
33  Midtown South     Korean Restaurant                 Hotel   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
33          Dessert Shop   Japanese Restaurant        Cosmetics Shop   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
33           Coffee Shop             Hotel Bar  Gym / Fitness Center   

   9th Most Common Venue 10th Most Common Venue  
33   American Restaurant            Karaoke Bar

Empty DataFrame
Columns: [Neighborhood, 1st Most Common Venue, 2nd Most Common Venue, 3rd Most Common Venue, 4th Most Common Venue, 5th Most Common Venue, 6th Most Common Venue, 7th Most Common Venue, 8th Most Common Venue, 9th Most Common Venue, 10th Most Common Venue]
Index: []

In [88]:
idx=[0,1,2,3,4,5]
for i in idx:
    display(Bronx_merged.loc[Bronx_merged['Cluster Labels'] ==i, Bronx_merged.columns[[1] + list(range(5, Bronx_merged.shape[1]))]])

Neighborhood      1st Most Common Venue   2nd Most Common Venue  \
0             Wakefield                   Pharmacy              Donut Shop   
2           Eastchester       Caribbean Restaurant           Deli / Bodega   
4             Riverdale                       Park             Bus Station   
5           Kingsbridge                Pizza Place                     Bar   
6              Woodlawn                        Pub             Pizza Place   
7               Norwood                Pizza Place                    Park   
8        Williamsbridge               Dance Studio               Nightclub   
9            Baychester                 Donut Shop          Baseball Field   
10       Pelham Parkway              Deli / Bodega      Italian Restaurant   
11          City Island         Seafood Restaurant  Thrift / Vintage Store   
12         Bedford Park                      Diner           Deli / Bodega   
13   University Heights       Fast Food Restaurant             Pizza Place   
15              Fordham             Clothing Store              Shoe Store   
16         East Tremont                Pizza Place            Dance Studio   
17           West Farms                Bus Station                    Park   
18         High  Bridge                   Pharmacy             Pizza Place   
19              Melrose                Pizza Place          Discount Store   
20           Mott Haven         Spanish Restaurant                     Gym   
21          Port Morris     Furniture / Home Store      Spanish Restaurant   
22             Longwood              Deli / Bodega          Sandwich Place   
23          Hunts Point                  Juice Bar      Seafood Restaurant   
24           Morrisania             Discount Store    Fast Food Restaurant   
26         Clason Point                       Park           Moving Target   
29          Parkchester                Supermarket             Pizza Place   
30   Westchester Square       Fast Food Restaurant      Mexican Restaurant   
32          Morris Park                Pizza Place                  Bakery   
34       Spuyten Duyvil                       Park          Tennis Stadium   
35      North Riverdale                Pizza Place      Italian Restaurant   
36           Pelham Bay       Fast Food Restaurant                    Bank   
37        Schuylerville         Mexican Restaurant             Pizza Place   
39          Castle Hill                   Pharmacy                    Bank   
40            Olinville       Caribbean Restaurant             Supermarket   
41       Pelham Gardens                Bus Station                Pharmacy   
43            Unionport  Latin American Restaurant          Ice Cream Shop   
46    Concourse Village             Sandwich Place             Pizza Place   
47           Mount Eden       Fast Food Restaurant                Pharmacy   
49            Bronxdale         Chinese Restaurant      Italian Restaurant   
50             Allerton                Pizza Place          Discount Store   
51  Kingsbridge Heights                Pizza Place      Mexican Restaurant   

              3rd Most Common Venue      4th Most Common Venue  \
0                        Laundromat       Caribbean Restaurant   
2                       Bus Station                      Diner   
4                             Plaza       Medical Supply Store   
5                    Sandwich Place  Latin American Restaurant   
6                     Deli / Bodega                 Food Truck   
7                     Deli / Bodega                   Pharmacy   
8                               Bar                 Soup Place   
9                              Bank                Bus Station   
10                      Pizza Place         Frozen Yogurt Shop   
11                  Harbor / Marina                        Bar   
12                      Pizza Place         Mexican Restaurant   
13                           Bakery        Fried Chicken Joint   
15                       Donut S

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
3    Fieldston           Music Venue                 River   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
3                 Plaza   Peruvian Restaurant             Nightclub   

  6th Most Common Venue  7th Most Common Venue 8th Most Common Venue  \
3                Office  Outdoors & Recreation          Outlet Store   

           9th Most Common Venue 10th Most Common Venue  
3  Paper / Office Supplies Store                   Park

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
28  Country Club        Sandwich Place         Deli / Bodega   

   3rd Most Common Venue          4th Most Common Venue 5th Most Common Venue  \
28            Playground  Vegetarian / Vegan Restaurant   Peruvian Restaurant   

   6th Most Common Venue 7th Most Common Venue  8th Most Common Venue  \
28             Nightclub                Office  Outdoors & Recreation   

   9th Most Common Venue         10th Most Common Venue  
28          Outlet Store  Paper / Office Supplies Store

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
1          Co-op City            Bagel Shop            Donut Shop   
14     Morris Heights         Grocery Store         Deli / Bodega   
25          Soundview    Chinese Restaurant        Discount Store   
42          Concourse         Grocery Store    Chinese Restaurant   
44           Edenwald           Supermarket                  Food   
45  Claremont Village         Grocery Store           Bus Station   
48         Mount Hope         Grocery Store         Deli / Bodega   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
1            Bus Station         Grocery Store              Pharmacy   
14    Spanish Restaurant           Pizza Place           Bus Station   
25          Burger Joint           Bus Station        Breakfast Spot   
42         Deli / Bodega     Convenience Store    Spanish Restaurant   
44        Sandwich Place           Fish Market         Grocery Store   
45    Chinese Restaurant                Bakery           Pizza Place   
48    Spanish Restaurant           Bus Station        Ice Cream Shop   

   6th Most Common Venue 7th Most Common Venue  8th Most Common Venue  \
1   Fast Food Restaurant           Pizza Place     Chinese Restaurant   
14              Pharmacy                  Bank  Performing Arts Venue   
25              Bus Stop         Grocery Store       Basketball Court   
42   Fried Chicken Joint           Supermarket   Caribbean Restaurant   
44    Athletics & Sports           Bus Station      Accessories Store   
45                  Food                   Gym   Caribbean Restaurant   
48     Mobile Phone Shop      Video Game Store            Pizza Place   

    9th Most Common Venue 10th Most Common Venue  
1                    Park     Salon / Barbershop  
14  Outdoors & Recreation           Outlet Store  
25               Pharmacy    Fried Chicken Joint  
42               Pharmacy            Pizza Place  
44  Performing Arts Venue  Outdoors & Recreation  
45                   Park           Liquor Store  
48             Donut Shop         Sandwich Place

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
27     Throgs Neck      Asian Restaurant           Coffee Shop   
31        Van Nest         Deli / Bodega           Pizza Place   
33         Belmont    Italian Restaurant           Pizza Place   
38  Edgewater Park    Italian Restaurant         Deli / Bodega   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
27   American Restaurant    Italian Restaurant         Deli / Bodega   
31            Food Truck           Bus Station        Cosmetics Shop   
33         Deli / Bodega                Bakery    Spanish Restaurant   
38    Chinese Restaurant           Pizza Place                   Pub   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
27            Sports Bar                   Bar           Pizza Place   
31            Donut Shop           Supermarket           Coffee Shop   
33          Dessert Shop         Grocery Store                  Bank   
38                   Bar            Sports Bar                  Park   

   9th Most Common Venue 10th Most Common Venue  
27     Mobile Phone Shop           Home Service  
31            Hookah Bar              BBQ Joint  
33           Coffee Shop                    Bar  
38             Bookstore            Coffee Shop

Empty DataFrame
Columns: [Neighborhood, 1st Most Common Venue, 2nd Most Common Venue, 3rd Most Common Venue, 4th Most Common Venue, 5th Most Common Venue, 6th Most Common Venue, 7th Most Common Venue, 8th Most Common Venue, 9th Most Common Venue, 10th Most Common Venue]
Index: []

In [89]:
idx=[0,1,2,3,4,5]
for i in idx:
    display(Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] ==i, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]])

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
28     Canarsie                   Gym      Asian Restaurant   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
28         Deli / Bodega       Thai Restaurant  Caribbean Restaurant   

   6th Most Common Venue 7th Most Common Venue  8th Most Common Venue  \
28          Home Service              Bus Line  Outdoors & Recreation   

   9th Most Common Venue 10th Most Common Venue  
28           Outdoor Gym      Other Repair Shop

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
21         Red Hook    Seafood Restaurant           Art Gallery   
31  Manhattan Beach                  Café              Bus Stop   
32     Coney Island      Baseball Stadium                  Park   
35    Dyker Heights          Dance Studio                  Park   
36  Gerritsen Beach        Ice Cream Shop                   Bar   
45     Bergen Beach                  Park       Harbor / Marina   
61     Fulton Ferry                  Park   American Restaurant   
65            Dumbo                  Park        Ice Cream Shop   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
21                  Park                   Bar            Bagel Shop   
31        Ice Cream Shop        Sandwich Place                 Beach   
32           Music Venue               Brewery          Skating Rink   
35          Burger Joint            Bagel Shop           Golf Course   
36           Pizza Place           Event Space          Liquor Store   
45        Baseball Field         Moving Target            Playground   
61        Ice Cream Shop             Roof Deck            Playground   
65           Coffee Shop                  Café        Scenic Lookout   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
21           Flower Shop                  Café                  Farm   
31                  Food            Playground       Harbor / Marina   
32   Monument / Landmark                  Café  Caribbean Restaurant   
35           Yoga Studio       Organic Grocery           Outdoor Gym   
36    Seafood Restaurant          Cocktail Bar         Boat or Ferry   
45    Athletics & Sports           Yoga Studio       Organic Grocery   
61                Bridge           Coffee Shop        Scenic Lookout   
65                Bakery           Yoga Studio                   Gym   

   9th Most Common Venue        10th Most Common Venue  
21           Pizza Place                       Brewery  
31     Other Repair Shop               Other Nightlife  
32                 Beach  Theme Park Ride / Attraction  
35     Other Repair Shop               Other Nightlife  
36       Harbor / Marina                    Bagel Shop  
45     Other Repair Shop               Other Nightlife  
61                Bakery                   Pizza Place  
65            Boxing Gym                     Bookstore

Neighborhood   1st Most Common Venue  \
0                   Bay Ridge                     Spa   
1                 Bensonhurst      Chinese Restaurant   
2                 Sunset Park      Mexican Restaurant   
3                  Greenpoint             Coffee Shop   
4                   Gravesend                  Bakery   
5              Brighton Beach      Russian Restaurant   
6              Sheepshead Bay      Turkish Restaurant   
7           Manhattan Terrace             Pizza Place   
8                    Flatbush           Deli / Bodega   
9               Crown Heights             Pizza Place   
10              East Flatbush      Chinese Restaurant   
11                 Kensington           Grocery Store   
12            Windsor Terrace           Deli / Bodega   
13           Prospect Heights                     Bar   
14                Brownsville     Fried Chicken Joint   
15               Williamsburg             Pizza Place   
16                   Bushwick                     Bar   
17         Bedford Stuyvesant                     Bar   
18           Brooklyn Heights           Deli / Bodega   
19                Cobble Hill             Coffee Shop   
20            Carroll Gardens      Italian Restaurant   
22                    Gowanus  Furniture / Home Store   
23                Fort Greene      Italian Restaurant   
24                 Park Slope             Coffee Shop   
25              Cypress Hills     Fried Chicken Joint   
26              East New York      Spanish Restaurant   
27              Starrett City             Bus Station   
29                  Flatlands                Pharmacy   
33                 Bath Beach             Pizza Place   
34               Borough Park                    Bank   
37                Marine Park           Deli / Bodega   
38               Clinton Hill      Italian Restaurant   
39                   Sea Gate                     Spa   
40                   Downtown             Coffee Shop   
41                Boerum Hill             Coffee Shop   
42  Prospect Lefferts Gardens                    Café   
43                 Ocean Hill           Deli / Bodega   
44                  City Line              Donut Shop   
46                    Midwood             Pizza Place   
47        Prospect Park South    Caribbean Restaurant   
48                 Georgetown                    Bank   
49          East Williamsburg                  Bakery   
50                 North Side             Coffee Shop   
51                 South Side                     Bar   
52              Ocean Parkway        Sushi Restaurant   
53              Fort Hamilton                Pharmacy   
54                Ditmas Park      Chinese Restaurant   
55                    Wingate     Fried Chicken Joint   
56                      Rugby           Grocery Store   
57             Remsen Village    Caribbean Restaurant   
58                   New Lots     Fried Chicken Joint   
60                 Mill Basin             Pizza Place   
62               Vinegar Hill              Food Truck   
63                 Weeksville            Liquor Store   
64          Broadway Junction             Gas Station   
66                  Homecrest                    Bank   
67              Highland Park           Deli / Bodega   
68                    Madison           Deli / Bodega   
69                    Erasmus    Caribbean Restaurant   

              2nd Most Common Venue 3rd Most Common Venue  \
0                Italian Restaurant           Pizza Place   
1                Italian Restaurant            Donut Shop   
2                              Bank           Pizza Place   
3                               Bar           Pizza Place   
4                       Pizza Place    Italian Restaurant   
5                        Donut Shop      Sushi Restaurant   
6                      Dessert Shop           Yoga Studio   
7                    Ice Cream Shop            Donut Shop   
8              Caribbean Restaurant           C

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
30  Mill Island             Locksmith                  Pool   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
30           Opera House           Outdoor Gym     Other Repair Shop   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
30       Other Nightlife  Other Great Outdoors       Organic Grocery   

   9th Most Common Venue 10th Most Common Venue  
30          Optical Shop            Yoga Studio

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
59  Paerdegat Basin              Bus Line                  Food   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
59          Home Service      Business Service          Optical Shop   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
59           Outdoor Gym     Other Repair Shop       Other Nightlife   

   9th Most Common Venue 10th Most Common Venue  
59  Other Great Outdoors        Organic Grocery

Empty DataFrame
Columns: [Neighborhood, 1st Most Common Venue, 2nd Most Common Venue, 3rd Most Common Venue, 4th Most Common Venue, 5th Most Common Venue, 6th Most Common Venue, 7th Most Common Venue, 8th Most Common Venue, 9th Most Common Venue, 10th Most Common Venue]
Index: []

In [90]:
idx=[0,1,2,3,4,5]
for i in idx:
    display(Queens_merged.loc[Queens_merged['Cluster Labels'] ==i, Queens_merged.columns[[1] + list(range(5, Queens_merged.shape[1]))]])

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
43  Breezy Point                 Beach   Monument / Landmark   
50      Neponsit                 Beach           Yoga Studio   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
43                 Trail           Yoga Studio             Nightclub   
50             Nightclub       Organic Grocery          Optical Shop   

   6th Most Common Venue    7th Most Common Venue    8th Most Common Venue  \
43          Optical Shop                   Office  North Indian Restaurant   
50                Office  North Indian Restaurant             Noodle House   

      9th Most Common Venue   10th Most Common Venue  
43             Noodle House  New American Restaurant  
50  New American Restaurant        Other Repair Shop

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
36       St. Albans  Caribbean Restaurant         Deli / Bodega   
39  Cambria Heights  Caribbean Restaurant        Cosmetics Shop   
52      Floral Park     Indian Restaurant      Basketball Court   
59        Laurelton  Caribbean Restaurant      Sculpture Garden   
63       Somerville                  Park       Nature Preserve   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
36          Dance Studio     Health Food Store            Donut Shop   
39            Restaurant              Pharmacy     Other Repair Shop   
52            Dosa Place         Grocery Store            Playground   
59                  Park            Playground         Train Station   
63           Yoga Studio             Nightclub       Organic Grocery   

   6th Most Common Venue 7th Most Common Venue    8th Most Common Venue  \
36     Convenience Store  Fast Food Restaurant       Seafood Restaurant   
39          Liquor Store             Nightclub       Chinese Restaurant   
52           Pizza Place            Hobby Shop          Organic Grocery   
59          Noodle House       Organic Grocery             Optical Shop   
63          Optical Shop                Office  North Indian Restaurant   

   9th Most Common Venue   10th Most Common Venue  
36    Chinese Restaurant            Shopping Mall  
39    Mexican Restaurant              Yoga Studio  
52          Optical Shop                   Office  
59                Office  North Indian Restaurant  
63          Noodle House  New American Restaurant

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
64   Brookville         Deli / Bodega           Yoga Studio   

      3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
64  New American Restaurant       Organic Grocery          Optical Shop   

   6th Most Common Venue    7th Most Common Venue 8th Most Common Venue  \
64                Office  North Indian Restaurant          Noodle House   

   9th Most Common Venue 10th Most Common Venue  
64             Nightclub        Nature Preserve

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
29    Fresh Meadows           Bus Station    Chinese Restaurant   
54  Jamaica Estates          Intersection           Bus Station   
70          Pomonok            Playground           Bus Station   
79        Bayswater          Tennis Court            Playground   

   3rd Most Common Venue    4th Most Common Venue 5th Most Common Venue  \
29              Pharmacy               Food Truck        Cosmetics Shop   
54             Nightclub          Other Nightlife       Organic Grocery   
70      Halal Restaurant           Scenic Lookout   Japanese Restaurant   
79           Yoga Studio  New American Restaurant          Optical Shop   

   6th Most Common Venue    7th Most Common Venue  \
29            Playground              Pizza Place   
54          Optical Shop                   Office   
70    Chinese Restaurant              Supermarket   
79                Office  North Indian Restaurant   

            8th Most Common Venue 9th Most Common Venue  \
29  Vegetarian / Vegan Restaurant         Grocery Store   
54        North Indian Restaurant          Noodle House   
70                    Pizza Place                   Bar   
79                   Noodle House             Nightclub   

     10th Most Common Venue  
29             Noodle House  
54  New American Restaurant  
70            Bowling Alley  
79          Nature Preserve

Neighborhood            1st Most Common Venue  \
0                Astoria                 Greek Restaurant   
1               Woodside                    Grocery Store   
2        Jackson Heights        Latin American Restaurant   
3               Elmhurst                  Thai Restaurant   
4           Howard Beach               Italian Restaurant   
5                 Corona               Mexican Restaurant   
6           Forest Hills             Gym / Fitness Center   
7            Kew Gardens               Chinese Restaurant   
8          Richmond Hill        Latin American Restaurant   
9               Flushing                Hotpot Restaurant   
10      Long Island City                            Hotel   
11             Sunnyside                      Pizza Place   
12         East Elmhurst                       Donut Shop   
13               Maspeth                      Pizza Place   
14             Ridgewood                    Deli / Bodega   
15              Glendale                          Brewery   
16             Rego Park                           Bakery   
17             Woodhaven                         Pharmacy   
18            Ozone Park                         Pharmacy   
19      South Ozone Park                    Deli / Bodega   
20         College Point                    Deli / Bodega   
21            Whitestone                    Deli / Bodega   
22               Bayside                              Bar   
23            Auburndale               Italian Restaurant   
24           Little Neck                      Pizza Place   
25            Douglaston                      Pizza Place   
26             Glen Oaks                         Pharmacy   
27             Bellerose                    Deli / Bodega   
28     Kew Gardens Hills                             Bank   
30             Briarwood                    Deli / Bodega   
31        Jamaica Center                   Clothing Store   
32       Oakland Gardens                Korean Restaurant   
33        Queens Village                         Bus Stop   
34                Hollis                   Baseball Field   
35         South Jamaica                      Bus Station   
37              Rochdale  Southern / Soul Food Restaurant   
38   Springfield Gardens                      Bus Station   
40              Rosedale                   Baseball Field   
41          Far Rockaway                      Pizza Place   
42         Broad Channel                         Dive Bar   
44              Steinway              Rental Car Location   
45            Beechhurst               Chinese Restaurant   
46           Bay Terrace                   Clothing Store   
47              Edgemere                    Metro Station   
48               Arverne                        Surf Spot   
49        Rockaway Beach                            Beach   
51           Murray Hill                Korean Restaurant   
53            Holliswood              Fried Chicken Joint   
55       Queensboro Hill               Chinese Restaurant   
56             Hillcrest         College Basketball Court   
57            Ravenswood              Fried Chicken Joint   
58            Lindenwood              Japanese Restaurant   
60           Lefrak City           Furniture / Home Store   
61          Belle Harbor                            Beach   
62         Rockaway Park                            Beach   
65              Bellaire               Italian Restaurant   
66          North Corona                    Deli / Bodega   
67  Forest Hills Gardens                    Grocery Store   
68         Jamaica Hills                         Pharmacy   
69                Utopia                    Deli / Bodega   
71       Astoria Heights                    Deli / Bodega   
72         Hunters Point                             Café   
73     Sunnyside Gardens                              Bar   
74            Blissville                       Donut Shop   
75               Roxbury                    Del

Empty DataFrame
Columns: [Neighborhood, 1st Most Common Venue, 2nd Most Common Venue, 3rd Most Common Venue, 4th Most Common Venue, 5th Most Common Venue, 6th Most Common Venue, 7th Most Common Venue, 8th Most Common Venue, 9th Most Common Venue, 10th Most Common Venue]
Index: []

In [92]:
idx=[0,1,2,3,4,5]
for i in idx:
    display(Staten_Island_merged.loc[Staten_Island_merged['Cluster Labels'] ==i, Staten_Island_merged.columns[[1] + list(range(5, Staten_Island_merged.shape[1]))]])

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
51  Emerson Hill         Moving Target                   ATM   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
51           Music Venue        Nightlife Spot          Optical Shop   

    6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
51  Outdoors & Recreation           Outlet Mall                  Park   

   9th Most Common Venue 10th Most Common Venue  
51   Peruvian Restaurant              Pet Store

Neighborhood 1st Most Common Venue        2nd Most Common Venue  \
0          St. George        Clothing Store                  Pizza Place   
2           Stapleton        Discount Store               Sandwich Place   
3            Rosebank    Italian Restaurant                Grocery Store   
4       West Brighton           Coffee Shop               Cosmetics Shop   
7         South Beach                  Pier                        Beach   
11  Castleton Corners           Pizza Place                         Bank   
12    New Springville           Pizza Place           Chinese Restaurant   
13             Travis                 Hotel                          Gym   
14           New Dorp    Italian Restaurant                  Pizza Place   
15            Oakwood                   Bar                Women's Store   
16        Great Kills           Pizza Place           Italian Restaurant   
17        Eltingville           Pizza Place             Sushi Restaurant   
18           Annadale           Pizza Place                          Bar   
19            Woodrow              Pharmacy             Sushi Restaurant   
21      Tompkinsville         Deli / Bodega                      Brewery   
22        Silver Lake   American Restaurant                 Burger Joint   
23          Sunnyside   American Restaurant                       Market   
25        Westerleigh                Arcade            Convenience Store   
26       Graniteville              Wine Bar                Boat or Ferry   
28           Arrochar         Deli / Bodega           Italian Restaurant   
29           Grasmere              Bus Stop                       Bakery   
30           Old Town    Italian Restaurant                Jewelry Store   
31       Dongan Hills              Pharmacy           Italian Restaurant   
32      Midland Beach                 Beach                Deli / Bodega   
33         Grant City  Fast Food Restaurant  Eastern European Restaurant   
34     New Dorp Beach    Italian Restaurant                Deli / Bodega   
35        Bay Terrace           Supermarket           Salon / Barbershop   
36           Huguenot                  Bank                Deli / Bodega   
37    Pleasant Plains            Donut Shop                  Pizza Place   
38       Butler Manor                  Pool               Baseball Field   
39         Charleston        Cosmetics Shop                  Coffee Shop   
40          Rossville           Pizza Place                   Bagel Shop   
41      Arden Heights         Deli / Bodega                   Playground   
42         Greenridge                 Diner                       Lawyer   
43  Heartland Village           Pizza Place                   Restaurant   
46         Bulls Head              Bus Stop                  Pizza Place   
47      Richmond Town    Italian Restaurant                          Spa   
48        Shore Acres              Bus Stop           Italian Restaurant   
49            Clifton    Mexican Restaurant                Grocery Store   
50            Concord         Deli / Bodega       Thrift / Vintage Store   
54           Elm Park         Deli / Bodega               Ice Cream Shop   
55      Manor Heights            Bagel Shop                 Liquor Store   
57       Sandy Ground              Bus Stop                 Intersection   
58        Egbertville    Italian Restaurant               Cosmetics Shop   
59       Prince's Bay           Pizza Place             Sushi Restaurant   
60    Lighthouse Hill    Italian Restaurant                   Art Museum   
61    Richmond Valley                  Bank   Construction & Landscaping   

         3rd Most Common Venue 4th Most Common Venue  \
0          Sporting Goods Shop    Italian Restaurant   
2                          Bar                  Bank   
3                 Intersection   Filipino Restaurant   
4                  Music Store    Italian Restaurant   
7                          ATM         Deli / Bodega   
11               Grocery

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
6       Todt Hill                  Park                   ATM   
44        Chelsea        Sandwich Place            Steakhouse   
52  Randall Manor                  Park            Playground   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
6             Restaurant            Nail Salon        Nightlife Spot   
44   Arts & Crafts Store                  Park                   ATM   
52            Bagel Shop                   ATM              Pharmacy   

   6th Most Common Venue  7th Most Common Venue 8th Most Common Venue  \
6           Optical Shop  Outdoors & Recreation           Outlet Mall   
44              Pharmacy         Nightlife Spot          Optical Shop   
52            Nail Salon         Nightlife Spot          Optical Shop   

    9th Most Common Venue 10th Most Common Venue  
6     Peruvian Restaurant              Pet Store  
44  Outdoors & Recreation            Outlet Mall  
52  Outdoors & Recreation            Outlet Mall

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
1       New Brighton         Deli / Bodega              Bus Stop   
5        Grymes Hill               Dog Run         Deli / Bodega   
8      Port Richmond              Bus Stop   Rental Car Location   
9   Mariner's Harbor         Deli / Bodega              Bus Stop   
20       Tottenville    Italian Restaurant    Mexican Restaurant   
24         Park Hill              Bus Stop           Coffee Shop   
27         Arlington         Deli / Bodega   American Restaurant   
45        Bloomfield     Recreation Center          Burger Joint   
56       Willowbrook              Bus Stop         Deli / Bodega   
62         Fox Hills              Bus Stop        Sandwich Place   

         3rd Most Common Venue 4th Most Common Venue   5th Most Common Venue  \
1                         Park           Bridal Shop          Discount Store   
5                  Pizza Place          Optical Shop   Outdoors & Recreation   
8   Construction & Landscaping   Martial Arts School                     ATM   
9           Athletics & Sports           Supermarket      Italian Restaurant   
20              Cosmetics Shop              Bus Stop  Thrift / Vintage Store   
24          Athletics & Sports                 Hotel    Gym / Fitness Center   
27               Boat or Ferry              Bus Stop             Coffee Shop   
45                    Bus Stop            Theme Park                     ATM   
56          Chinese Restaurant            Bagel Shop             Pizza Place   
62                         ATM                  Pier          Nightlife Spot   

   6th Most Common Venue  7th Most Common Venue  8th Most Common Venue  \
1             Playground          Bowling Alley                 Market   
5            Outlet Mall                   Park    Peruvian Restaurant   
8            Pizza Place  Outdoors & Recreation            Outlet Mall   
9                 Lawyer            Pizza Place  Outdoors & Recreation   
20         Deli / Bodega           Liquor Store  Outdoors & Recreation   
24                   ATM                   Pier           Optical Shop   
27         Grocery Store                   Pier  Outdoors & Recreation   
45              Pharmacy           Optical Shop  Outdoors & Recreation   
56            Playground  Outdoors & Recreation            Outlet Mall   
62          Optical Shop  Outdoors & Recreation            Outlet Mall   

    9th Most Common Venue 10th Most Common Venue  
1     Martial Arts School           Liquor Store  
5               Pet Store               Pharmacy  
8                    Park    Peruvian Restaurant  
9             Outlet Mall              Juice Bar  
20                   Park             Kids Store  
24  Outdoors & Recreation            Outlet Mall  
27            Outlet Mall                   Park  
45            Outlet Mall                   Park  
56                   Park    Peruvian Restaurant  
62                   Park    Peruvian Restaurant

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
10   Port Ivory                   Bar                   ATM   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
10              Pharmacy            Nail Salon        Nightlife Spot   

   6th Most Common Venue  7th Most Common Venue 8th Most Common Venue  \
10          Optical Shop  Outdoors & Recreation           Outlet Mall   

   9th Most Common Venue 10th Most Common Venue  
10                  Park    Peruvian Restaurant

Empty DataFrame
Columns: [Neighborhood, 1st Most Common Venue, 2nd Most Common Venue, 3rd Most Common Venue, 4th Most Common Venue, 5th Most Common Venue, 6th Most Common Venue, 7th Most Common Venue, 8th Most Common Venue, 9th Most Common Venue, 10th Most Common Venue]
Index: []

### Results and Discussion
It is obvious that there are a great number of restaurants in New York City , mostly surrounded in city areas. Manhattan and Brooklyn seems to have greater variety of cusines and at the same time, most commonly crowded venues too.

After directing our attention to New York City we first created a map of New York City Boroughs. Then each Borough was analyzed to see most common places.After finding latitude and longitudes of each Borough , we ran  k-means to cluster the neighborhood into 5 clusters for each Borough and examined each cluster. Then top 10 common venues were identified and listed.This helps to identify what are the places having most restaurants and most densed venues. 

The data reveals that Manhattan and Brooklyn have most restaurants and varienty of cusines. Also it has most crowded venues. Staten Island seem to have less number of restaurants and so it may not be a suitable place for restaurant.

### Conclusion
By Analyzing data , it is clear that Manhattan and Brooklyn are the most busy place and have larger scope for restaurant. Also Staten Island seems to be the least busy place and starting a restaurant may not be a good choice over there.